# Ordering by metrics and retraining phase

## Dataset: GTSRB

## Experiment configuration 1


## Configuration 1
	1. Incremental guided retraining starting from scratch using the new adversarial inputs and original training set

In [ ]:
pip install --user tensorflow==2.5

In [1]:
import argparse

import numpy as np
import tensorflow as tf
import keras.backend as K

import matplotlib.pyplot as plt

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.regularizers import l2

import glob
import os
import cv2
import keras


import time
import argparse

from tqdm import tqdm

from keras.models import load_model, Model


In [2]:
tf.__version__ == '2.5.0' # Version of tensorflow

True

In [3]:
cd C:/Users/fjdur/Documents/utils/

C:\Users\fjdur\Documents\utils


In [4]:
# utils for project
import utils_guided_retraining as utils

2.5.0
keras
2.5.0


In [5]:
#cd '../notebooks/'

In [6]:
dataset = "gtsrb"

In [7]:
x_train,y_train = utils.get_data(dataset,"Train")
x_val,y_val = utils.get_data(dataset,"Val")
x_test,y_test = utils.get_data(dataset,"Test")

In [8]:
x_train_and_adversary,y_train_and_adversary = utils.get_data(dataset,"Train_and_adversary")

In [9]:
x_adversary_training = x_train_and_adversary[len(x_train):]
print(len(x_adversary_training))
y_adversary_training = y_train_and_adversary[len(y_train):]

print(len(y_adversary_training))

3921
3921


In [10]:
# Obtaining adversarial examples for testing 
x_test_and_adversary,y_test_and_adversary = utils.get_adversarial_data(dataset,'Test_adversarial') 

In [11]:
x_adversary_test_fgsm = x_test_and_adversary[len(x_test):]
print(len(x_adversary_test_fgsm))
y_adversary_test_fgsm = y_test_and_adversary[len(y_test):]

print(len(y_adversary_test_fgsm))

3921
3921


## ----

In [12]:
# Original model 
model_dir = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06/"


In [13]:
model_original = utils.My_model(dataset,True, model_dir)

Model loaded correctly


In [14]:
n_data_points = 1750


In [15]:
import gc

In [14]:
lsa_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/lsa_values.npy"
dsa_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/dsa_values.npy"

## Loading LSA and DSA values 

In [16]:
lsa_values = np.load(lsa_direction)


In [17]:
dsa_values = np.load(dsa_direction)

In [18]:

# Obtaining top n images by LSA values
top_images_by_lsa = utils.get_x_of_indexes(list(np.flip(np.argsort(lsa_values))),x_train_and_adversary)
top_labels_by_lsa = utils.get_x_of_indexes(list(np.flip(np.argsort(lsa_values))),y_train_and_adversary)


In [19]:

top_images_by_dsa = utils.get_x_of_indexes(list(np.flip(np.argsort(dsa_values))),x_train_and_adversary)
top_labels_by_dsa = utils.get_x_of_indexes(list(np.flip(np.argsort(dsa_values))),y_train_and_adversary)


In [20]:
print(x_train_and_adversary.shape)
len(top_images_by_lsa)//20

(35287, 48, 48, 1)


1764

In [21]:
m = 1750 
n = 0
image_sets_lsa = []
label_sets_lsa = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_lsa)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_lsa)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_lsa)%m))
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m])
    image_sets_lsa.append(top_images_by_lsa_n)
    label_sets_lsa.append(top_labels_by_lsa_n)
    print(len(top_images_by_lsa_n))
    n += m

0 :
0  ->  1750
1750
1 :
0  ->  3500
3500
2 :
0  ->  5250
5250
3 :
0  ->  7000
7000
4 :
0  ->  8750
8750
5 :
0  ->  10500
10500
6 :
0  ->  12250
12250
7 :
0  ->  14000
14000
8 :
0  ->  15750
15750
9 :
0  ->  17500
17500
10 :
0  ->  19250
19250
11 :
0  ->  21000
21000
12 :
0  ->  22750
22750
13 :
0  ->  24500
24500
14 :
0  ->  26250
26250
15 :
0  ->  28000
28000
16 :
0  ->  29750
29750
17 :
0  ->  31500
31500
18 :
0  ->  33250
33250
19 :
0  ->  35000
35000
20 :
Last
0  ->  37037
35287


In [24]:
len(image_sets_lsa[-1])

35287

In [22]:
m = 1750
n = 0
image_sets_lsa = []
label_sets_lsa = []


for i in range(len(top_images_by_lsa)//m):
    print(i,":")
    if (i+1 >= len(top_images_by_lsa)//m):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_lsa)%m))
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m])
    image_sets_lsa.append(top_images_by_lsa_n)
    label_sets_lsa.append(top_labels_by_lsa_n)
    print(len(top_images_by_lsa_n))
    n += m

print(len(image_sets_lsa)," total sets")

0 :
0  ->  1750
1750
1 :
0  ->  3500
3500
2 :
0  ->  5250
5250
3 :
0  ->  7000
7000
4 :
0  ->  8750
8750
5 :
0  ->  10500
10500
6 :
0  ->  12250
12250
7 :
0  ->  14000
14000
8 :
0  ->  15750
15750
9 :
0  ->  17500
17500
10 :
0  ->  19250
19250
11 :
0  ->  21000
21000
12 :
0  ->  22750
22750
13 :
0  ->  24500
24500
14 :
0  ->  26250
26250
15 :
0  ->  28000
28000
16 :
0  ->  29750
29750
17 :
0  ->  31500
31500
18 :
0  ->  33250
33250
19 :
Last
0  ->  35287
35287
20  total sets


## Training guided by LSA

In [22]:
len(label_sets_lsa)

21

In [23]:
print(model_dir)

models_lsa = []
for i in range(len(label_sets_lsa)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_lsa.append(model)


C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06/
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled
20 :
Model loaded correctly
Model compiled


In [24]:
n = 0
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n=n+1

Epoch 1/10
28/28 [==============================] - 75s 880ms/step - loss: 2.0385 - accuracy: 0.5040 - val_loss: 0.0987 - val_accuracy: 0.9816
Epoch 2/10
28/28 [==============================] - 22s 787ms/step - loss: 1.2656 - accuracy: 0.6469 - val_loss: 0.1927 - val_accuracy: 0.9712
Epoch 3/10
28/28 [==============================] - 22s 797ms/step - loss: 1.0187 - accuracy: 0.7009 - val_loss: 0.0976 - val_accuracy: 0.9755
Epoch 4/10
28/28 [==============================] - 23s 848ms/step - loss: 0.8468 - accuracy: 0.7524 - val_loss: 0.1290 - val_accuracy: 0.9701
Epoch 5/10
28/28 [==============================] - 17s 595ms/step - loss: 0.7409 - accuracy: 0.7888 - val_loss: 0.1589 - val_accuracy: 0.9548
Epoch 6/10
28/28 [==============================] - 17s 630ms/step - loss: 0.6606 - accuracy: 0.8026 - val_loss: 0.1175 - val_accuracy: 0.9635
Epoch 7/10
28/28 [==============================] - 17s 621ms/step - loss: 0.6041 - accuracy: 0.8308 - val_loss: 0.1376 - val_accuracy: 0.9625

In [25]:
n=1
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

Epoch 1/10
55/55 [==============================] - 31s 499ms/step - loss: 1.1640 - accuracy: 0.7321 - val_loss: 0.0923 - val_accuracy: 0.9834
Epoch 2/10
55/55 [==============================] - 27s 492ms/step - loss: 0.6705 - accuracy: 0.8111 - val_loss: 0.0893 - val_accuracy: 0.9722
Epoch 3/10
55/55 [==============================] - 29s 529ms/step - loss: 0.5537 - accuracy: 0.8434 - val_loss: 0.1042 - val_accuracy: 0.9737
Epoch 4/10
55/55 [==============================] - 26s 471ms/step - loss: 0.4487 - accuracy: 0.8729 - val_loss: 0.1438 - val_accuracy: 0.9643
Epoch 5/10
55/55 [==============================] - 26s 482ms/step - loss: 0.3669 - accuracy: 0.8991 - val_loss: 0.0650 - val_accuracy: 0.9816
Epoch 6/10
55/55 [==============================] - 29s 529ms/step - loss: 0.3717 - accuracy: 0.8897 - val_loss: 0.0937 - val_accuracy: 0.9727
Epoch 7/10
55/55 [==============================] - 28s 518ms/step - loss: 0.2959 - accuracy: 0.9178 - val_loss: 0.1493 - val_accuracy: 0.9584

In [26]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

2
Epoch 1/10
83/83 [==============================] - 39s 434ms/step - loss: 0.7418 - accuracy: 0.8188 - val_loss: 0.1315 - val_accuracy: 0.9678
Epoch 2/10
83/83 [==============================] - 36s 437ms/step - loss: 0.4966 - accuracy: 0.8663 - val_loss: 0.0974 - val_accuracy: 0.9727
Epoch 3/10
83/83 [==============================] - 37s 442ms/step - loss: 0.3719 - accuracy: 0.8940 - val_loss: 0.1039 - val_accuracy: 0.9742
Epoch 4/10
83/83 [==============================] - 37s 445ms/step - loss: 0.3057 - accuracy: 0.9138 - val_loss: 0.1060 - val_accuracy: 0.9740
Epoch 5/10
83/83 [==============================] - 37s 449ms/step - loss: 0.2678 - accuracy: 0.9183 - val_loss: 0.0968 - val_accuracy: 0.9758
Epoch 6/10
83/83 [==============================] - 38s 458ms/step - loss: 0.2629 - accuracy: 0.9247 - val_loss: 0.2320 - val_accuracy: 0.9469
Epoch 7/10
83/83 [==============================] - 38s 463ms/step - loss: 0.2417 - accuracy: 0.9314 - val_loss: 0.1569 - val_accuracy: 0.95

In [27]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

3
Epoch 1/10
110/110 [==============================] - 55s 474ms/step - loss: 0.6034 - accuracy: 0.8577 - val_loss: 0.0938 - val_accuracy: 0.9778
Epoch 2/10
110/110 [==============================] - 58s 526ms/step - loss: 0.3751 - accuracy: 0.9001 - val_loss: 0.0755 - val_accuracy: 0.9788
Epoch 3/10
110/110 [==============================] - 53s 485ms/step - loss: 0.2963 - accuracy: 0.9176 - val_loss: 0.0899 - val_accuracy: 0.9752
Epoch 4/10
110/110 [==============================] - 52s 472ms/step - loss: 0.2697 - accuracy: 0.9246 - val_loss: 0.0951 - val_accuracy: 0.9742
Epoch 5/10
110/110 [==============================] - 58s 524ms/step - loss: 0.2247 - accuracy: 0.9363 - val_loss: 0.0936 - val_accuracy: 0.9755
Epoch 6/10
110/110 [==============================] - 55s 496ms/step - loss: 0.1947 - accuracy: 0.9431 - val_loss: 0.0770 - val_accuracy: 0.9770
Epoch 7/10
110/110 [==============================] - 56s 505ms/step - loss: 0.1884 - accuracy: 0.9414 - val_loss: 0.0996 - val_

In [28]:
print(n) #4
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

4
Epoch 1/10
137/137 [==============================] - 66s 464ms/step - loss: 0.5064 - accuracy: 0.8709 - val_loss: 0.0562 - val_accuracy: 0.9870
Epoch 2/10
137/137 [==============================] - 63s 460ms/step - loss: 0.3342 - accuracy: 0.9092 - val_loss: 0.0735 - val_accuracy: 0.9837
Epoch 3/10
137/137 [==============================] - 63s 463ms/step - loss: 0.2687 - accuracy: 0.9261 - val_loss: 0.1000 - val_accuracy: 0.9691
Epoch 4/10
137/137 [==============================] - 69s 504ms/step - loss: 0.2102 - accuracy: 0.9413 - val_loss: 0.0616 - val_accuracy: 0.9821
Epoch 5/10
137/137 [==============================] - 69s 503ms/step - loss: 0.1974 - accuracy: 0.9428 - val_loss: 0.0914 - val_accuracy: 0.9747
Epoch 6/10
137/137 [==============================] - 69s 504ms/step - loss: 0.1808 - accuracy: 0.9489 - val_loss: 0.0824 - val_accuracy: 0.9778
Epoch 7/10
137/137 [==============================] - 69s 502ms/step - loss: 0.1612 - accuracy: 0.9521 - val_loss: 0.0996 - val_

In [29]:
n=5
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

5
Epoch 1/10
165/165 [==============================] - 85s 493ms/step - loss: 0.4530 - accuracy: 0.8944 - val_loss: 0.0340 - val_accuracy: 0.9921
Epoch 2/10
165/165 [==============================] - 82s 498ms/step - loss: 0.2641 - accuracy: 0.9265 - val_loss: 0.1003 - val_accuracy: 0.9791
Epoch 3/10
165/165 [==============================] - 82s 497ms/step - loss: 0.2134 - accuracy: 0.9403 - val_loss: 0.0602 - val_accuracy: 0.9844
Epoch 4/10
165/165 [==============================] - 82s 499ms/step - loss: 0.1968 - accuracy: 0.9467 - val_loss: 0.1064 - val_accuracy: 0.9735
Epoch 5/10
165/165 [==============================] - 81s 492ms/step - loss: 0.1799 - accuracy: 0.9500 - val_loss: 0.1061 - val_accuracy: 0.9691
Epoch 6/10
165/165 [==============================] - 82s 499ms/step - loss: 0.1573 - accuracy: 0.9571 - val_loss: 0.0637 - val_accuracy: 0.9844
Epoch 7/10
165/165 [==============================] - 82s 496ms/step - loss: 0.1481 - accuracy: 0.9581 - val_loss: 0.2386 - val_

In [30]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

6
Epoch 1/10
192/192 [==============================] - 98s 492ms/step - loss: 0.3784 - accuracy: 0.9062 - val_loss: 0.0596 - val_accuracy: 0.9872
Epoch 2/10
192/192 [==============================] - 94s 488ms/step - loss: 0.2303 - accuracy: 0.9374 - val_loss: 0.0603 - val_accuracy: 0.9855
Epoch 3/10
192/192 [==============================] - 94s 490ms/step - loss: 0.2152 - accuracy: 0.9398 - val_loss: 0.0824 - val_accuracy: 0.9786
Epoch 4/10
192/192 [==============================] - 94s 491ms/step - loss: 0.1978 - accuracy: 0.9499 - val_loss: 0.1259 - val_accuracy: 0.9699
Epoch 5/10
192/192 [==============================] - 94s 490ms/step - loss: 0.1652 - accuracy: 0.9569 - val_loss: 0.0903 - val_accuracy: 0.9775
Epoch 6/10
192/192 [==============================] - 96s 502ms/step - loss: 0.1476 - accuracy: 0.9618 - val_loss: 0.0891 - val_accuracy: 0.9770
Epoch 7/10
192/192 [==============================] - 95s 496ms/step - loss: 0.1402 - accuracy: 0.9606 - val_loss: 0.0742 - val_

In [31]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

7
Epoch 1/10
219/219 [==============================] - 112s 494ms/step - loss: 0.3393 - accuracy: 0.9165 - val_loss: 0.1129 - val_accuracy: 0.9852
Epoch 2/10
219/219 [==============================] - 107s 490ms/step - loss: 0.2203 - accuracy: 0.9416 - val_loss: 0.0513 - val_accuracy: 0.9880
Epoch 3/10
219/219 [==============================] - 106s 486ms/step - loss: 0.1919 - accuracy: 0.9481 - val_loss: 0.0544 - val_accuracy: 0.9883
Epoch 4/10
219/219 [==============================] - 106s 484ms/step - loss: 0.1612 - accuracy: 0.9556 - val_loss: 0.0558 - val_accuracy: 0.9865
Epoch 5/10
219/219 [==============================] - 109s 499ms/step - loss: 0.1563 - accuracy: 0.9565 - val_loss: 0.1859 - val_accuracy: 0.9500
Epoch 6/10
219/219 [==============================] - 108s 493ms/step - loss: 0.1468 - accuracy: 0.9608 - val_loss: 0.0827 - val_accuracy: 0.9829
Epoch 7/10
219/219 [==============================] - 106s 485ms/step - loss: 0.1463 - accuracy: 0.9599 - val_loss: 0.0821

In [32]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

8
Epoch 1/10
247/247 [==============================] - 124s 489ms/step - loss: 0.3022 - accuracy: 0.9257 - val_loss: 0.0464 - val_accuracy: 0.9893
Epoch 2/10
247/247 [==============================] - 122s 495ms/step - loss: 0.2075 - accuracy: 0.9469 - val_loss: 0.0469 - val_accuracy: 0.9888
Epoch 3/10
247/247 [==============================] - 123s 499ms/step - loss: 0.1730 - accuracy: 0.9535 - val_loss: 0.0549 - val_accuracy: 0.9857
Epoch 4/10
247/247 [==============================] - 126s 512ms/step - loss: 0.1687 - accuracy: 0.9550 - val_loss: 0.0595 - val_accuracy: 0.9811
Epoch 5/10
247/247 [==============================] - 119s 482ms/step - loss: 0.1440 - accuracy: 0.9614 - val_loss: 0.0550 - val_accuracy: 0.9855
Epoch 6/10
247/247 [==============================] - 119s 482ms/step - loss: 0.1418 - accuracy: 0.9628 - val_loss: 0.0671 - val_accuracy: 0.9839
Epoch 7/10
247/247 [==============================] - 119s 483ms/step - loss: 0.1362 - accuracy: 0.9632 - val_loss: 0.1242

In [33]:
print(n)#9
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

9
Epoch 1/10
274/274 [==============================] - 136s 484ms/step - loss: 0.2901 - accuracy: 0.9304 - val_loss: 0.0382 - val_accuracy: 0.9906
Epoch 2/10
274/274 [==============================] - 132s 481ms/step - loss: 0.1944 - accuracy: 0.9489 - val_loss: 0.0587 - val_accuracy: 0.9855
Epoch 3/10
274/274 [==============================] - 132s 482ms/step - loss: 0.1609 - accuracy: 0.9581 - val_loss: 0.0692 - val_accuracy: 0.9837
Epoch 4/10
274/274 [==============================] - 132s 482ms/step - loss: 0.1634 - accuracy: 0.9579 - val_loss: 0.1521 - val_accuracy: 0.9533
Epoch 5/10
274/274 [==============================] - 132s 483ms/step - loss: 0.1362 - accuracy: 0.9643 - val_loss: 0.1057 - val_accuracy: 0.9768
Epoch 6/10
274/274 [==============================] - 132s 482ms/step - loss: 0.1374 - accuracy: 0.9633 - val_loss: 0.0773 - val_accuracy: 0.9786
Epoch 7/10
274/274 [==============================] - 133s 484ms/step - loss: 0.1228 - accuracy: 0.9676 - val_loss: 0.1644

In [34]:
## print(n)
n=10
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

Epoch 1/10
301/301 [==============================] - 151s 488ms/step - loss: 0.2741 - accuracy: 0.9342 - val_loss: 0.0219 - val_accuracy: 0.9929
Epoch 2/10
301/301 [==============================] - 145s 482ms/step - loss: 0.1847 - accuracy: 0.9525 - val_loss: 0.0607 - val_accuracy: 0.9870
Epoch 3/10
301/301 [==============================] - 145s 481ms/step - loss: 0.1570 - accuracy: 0.9590 - val_loss: 0.0628 - val_accuracy: 0.9870
Epoch 4/10
301/301 [==============================] - 145s 483ms/step - loss: 0.1380 - accuracy: 0.9639 - val_loss: 0.0757 - val_accuracy: 0.9821
Epoch 5/10
301/301 [==============================] - 145s 480ms/step - loss: 0.1293 - accuracy: 0.9664 - val_loss: 0.0784 - val_accuracy: 0.9832
Epoch 6/10
301/301 [==============================] - 144s 480ms/step - loss: 0.1349 - accuracy: 0.9660 - val_loss: 0.1230 - val_accuracy: 0.9722
Epoch 7/10
301/301 [==============================] - 145s 480ms/step - loss: 0.1325 - accuracy: 0.9665 - val_loss: 0.1111 -

In [35]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

11
Epoch 1/10
329/329 [==============================] - 160s 476ms/step - loss: 0.2567 - accuracy: 0.9380 - val_loss: 0.0584 - val_accuracy: 0.9837
Epoch 2/10
329/329 [==============================] - 157s 478ms/step - loss: 0.1665 - accuracy: 0.9540 - val_loss: 0.0636 - val_accuracy: 0.9893
Epoch 3/10
329/329 [==============================] - 157s 478ms/step - loss: 0.1546 - accuracy: 0.9589 - val_loss: 0.1265 - val_accuracy: 0.9804
Epoch 4/10
329/329 [==============================] - 157s 477ms/step - loss: 0.1469 - accuracy: 0.9622 - val_loss: 0.1211 - val_accuracy: 0.9709
Epoch 5/10
329/329 [==============================] - 157s 477ms/step - loss: 0.1280 - accuracy: 0.9671 - val_loss: 0.0694 - val_accuracy: 0.9860
Epoch 6/10
329/329 [==============================] - 157s 477ms/step - loss: 0.1258 - accuracy: 0.9661 - val_loss: 0.2135 - val_accuracy: 0.9650
Epoch 7/10
329/329 [==============================] - 157s 477ms/step - loss: 0.1295 - accuracy: 0.9691 - val_loss: 0.142

In [36]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

12
Epoch 1/10
356/356 [==============================] - 178s 489ms/step - loss: 0.2600 - accuracy: 0.9429 - val_loss: 0.0538 - val_accuracy: 0.9865
Epoch 2/10
356/356 [==============================] - 170s 476ms/step - loss: 0.1506 - accuracy: 0.9602 - val_loss: 0.1052 - val_accuracy: 0.9781
Epoch 3/10
356/356 [==============================] - 169s 476ms/step - loss: 0.1600 - accuracy: 0.9598 - val_loss: 0.1500 - val_accuracy: 0.9699
Epoch 4/10
356/356 [==============================] - 170s 478ms/step - loss: 0.1367 - accuracy: 0.9638 - val_loss: 0.0986 - val_accuracy: 0.9786
Epoch 5/10
356/356 [==============================] - 172s 483ms/step - loss: 0.1361 - accuracy: 0.9670 - val_loss: 0.0670 - val_accuracy: 0.9824
Epoch 6/10
356/356 [==============================] - 169s 476ms/step - loss: 0.1318 - accuracy: 0.9677 - val_loss: 0.1190 - val_accuracy: 0.9709
Epoch 7/10
356/356 [==============================] - 170s 477ms/step - loss: 0.1262 - accuracy: 0.9712 - val_loss: 0.119

In [37]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

13
Epoch 1/10
383/383 [==============================] - 186s 477ms/step - loss: 0.2216 - accuracy: 0.9486 - val_loss: 0.0447 - val_accuracy: 0.9913
Epoch 2/10
383/383 [==============================] - 184s 481ms/step - loss: 0.1592 - accuracy: 0.9580 - val_loss: 0.0876 - val_accuracy: 0.9814
Epoch 3/10
383/383 [==============================] - 182s 475ms/step - loss: 0.1330 - accuracy: 0.9650 - val_loss: 0.0794 - val_accuracy: 0.9747
Epoch 4/10
383/383 [==============================] - 184s 479ms/step - loss: 0.1258 - accuracy: 0.9685 - val_loss: 0.1208 - val_accuracy: 0.9663
Epoch 5/10
383/383 [==============================] - 182s 476ms/step - loss: 0.1252 - accuracy: 0.9688 - val_loss: 0.0800 - val_accuracy: 0.9809
Epoch 6/10
383/383 [==============================] - 183s 478ms/step - loss: 0.1229 - accuracy: 0.9706 - val_loss: 0.1157 - val_accuracy: 0.9732
Epoch 7/10
383/383 [==============================] - 182s 475ms/step - loss: 0.1322 - accuracy: 0.9697 - val_loss: 0.199

In [38]:
print(n)#14
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

14
Epoch 1/10
411/411 [==============================] - 199s 475ms/step - loss: 0.2164 - accuracy: 0.9506 - val_loss: 0.0519 - val_accuracy: 0.9883
Epoch 2/10
411/411 [==============================] - 198s 481ms/step - loss: 0.1452 - accuracy: 0.9625 - val_loss: 0.0474 - val_accuracy: 0.9867
Epoch 3/10
411/411 [==============================] - 195s 474ms/step - loss: 0.1430 - accuracy: 0.9630 - val_loss: 0.1147 - val_accuracy: 0.9778
Epoch 4/10
411/411 [==============================] - 196s 477ms/step - loss: 0.1420 - accuracy: 0.9638 - val_loss: 0.0686 - val_accuracy: 0.9847
Epoch 5/10
411/411 [==============================] - 197s 478ms/step - loss: 0.1308 - accuracy: 0.9683 - val_loss: 0.0912 - val_accuracy: 0.9775
Epoch 6/10
411/411 [==============================] - 195s 475ms/step - loss: 0.1178 - accuracy: 0.9709 - val_loss: 0.1416 - val_accuracy: 0.9709
Epoch 7/10
411/411 [==============================] - 195s 474ms/step - loss: 0.1431 - accuracy: 0.9686 - val_loss: 0.120

In [39]:
n=15
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

15
Epoch 1/10
438/438 [==============================] - 211s 474ms/step - loss: 0.2081 - accuracy: 0.9499 - val_loss: 0.0524 - val_accuracy: 0.9903
Epoch 2/10
438/438 [==============================] - 208s 475ms/step - loss: 0.1529 - accuracy: 0.9612 - val_loss: 0.0585 - val_accuracy: 0.9849
Epoch 3/10
438/438 [==============================] - 208s 475ms/step - loss: 0.1284 - accuracy: 0.9653 - val_loss: 0.1069 - val_accuracy: 0.9717
Epoch 4/10
438/438 [==============================] - 208s 475ms/step - loss: 0.1329 - accuracy: 0.9669 - val_loss: 0.0566 - val_accuracy: 0.9862
Epoch 5/10
438/438 [==============================] - 208s 475ms/step - loss: 0.1367 - accuracy: 0.9657 - val_loss: 0.0691 - val_accuracy: 0.9878
Epoch 6/10
438/438 [==============================] - 207s 473ms/step - loss: 0.1335 - accuracy: 0.9691 - val_loss: 0.0915 - val_accuracy: 0.9788
Epoch 7/10
438/438 [==============================] - 207s 472ms/step - loss: 0.1247 - accuracy: 0.9697 - val_loss: 0.115

In [40]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

16
Epoch 1/10
465/465 [==============================] - 224s 474ms/step - loss: 0.2353 - accuracy: 0.9507 - val_loss: 0.0608 - val_accuracy: 0.9839
Epoch 2/10
465/465 [==============================] - 222s 477ms/step - loss: 0.1431 - accuracy: 0.9636 - val_loss: 0.0531 - val_accuracy: 0.9878
Epoch 3/10
465/465 [==============================] - 222s 478ms/step - loss: 0.1276 - accuracy: 0.9671 - val_loss: 0.0729 - val_accuracy: 0.9824
Epoch 4/10
465/465 [==============================] - 221s 475ms/step - loss: 0.1314 - accuracy: 0.9677 - val_loss: 0.0599 - val_accuracy: 0.9867
Epoch 5/10
465/465 [==============================] - 220s 474ms/step - loss: 0.1240 - accuracy: 0.9698 - val_loss: 0.0392 - val_accuracy: 0.9929
Epoch 6/10
465/465 [==============================] - 220s 473ms/step - loss: 0.1206 - accuracy: 0.9713 - val_loss: 0.0933 - val_accuracy: 0.9724
Epoch 7/10
465/465 [==============================] - 220s 474ms/step - loss: 0.1226 - accuracy: 0.9710 - val_loss: 0.125

In [41]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

17
Epoch 1/10
493/493 [==============================] - 237s 475ms/step - loss: 0.1934 - accuracy: 0.9563 - val_loss: 0.0686 - val_accuracy: 0.9824
Epoch 2/10
493/493 [==============================] - 233s 473ms/step - loss: 0.1461 - accuracy: 0.9640 - val_loss: 0.1028 - val_accuracy: 0.9804
Epoch 3/10
493/493 [==============================] - 233s 472ms/step - loss: 0.1408 - accuracy: 0.9636 - val_loss: 0.0345 - val_accuracy: 0.9918
Epoch 4/10
493/493 [==============================] - 233s 472ms/step - loss: 0.1376 - accuracy: 0.9661 - val_loss: 0.0545 - val_accuracy: 0.9885
Epoch 5/10
493/493 [==============================] - 235s 477ms/step - loss: 0.1263 - accuracy: 0.9704 - val_loss: 0.0908 - val_accuracy: 0.9834
Epoch 6/10
493/493 [==============================] - 233s 473ms/step - loss: 0.1297 - accuracy: 0.9704 - val_loss: 0.1480 - val_accuracy: 0.9740
Epoch 7/10
493/493 [==============================] - 233s 473ms/step - loss: 0.1251 - accuracy: 0.9700 - val_loss: 0.129

In [42]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

18
Epoch 1/10
520/520 [==============================] - 249s 472ms/step - loss: 0.1827 - accuracy: 0.9563 - val_loss: 0.0344 - val_accuracy: 0.9923
Epoch 2/10
520/520 [==============================] - 245s 472ms/step - loss: 0.1435 - accuracy: 0.9628 - val_loss: 0.0287 - val_accuracy: 0.9926
Epoch 3/10
520/520 [==============================] - 246s 473ms/step - loss: 0.1323 - accuracy: 0.9661 - val_loss: 0.0282 - val_accuracy: 0.9954
Epoch 4/10
520/520 [==============================] - 246s 473ms/step - loss: 0.1284 - accuracy: 0.9668 - val_loss: 0.0294 - val_accuracy: 0.9913
Epoch 5/10
520/520 [==============================] - 246s 473ms/step - loss: 0.1208 - accuracy: 0.9705 - val_loss: 0.0502 - val_accuracy: 0.9903
Epoch 6/10
520/520 [==============================] - 246s 473ms/step - loss: 0.1300 - accuracy: 0.9700 - val_loss: 0.0238 - val_accuracy: 0.9929
Epoch 7/10
520/520 [==============================] - 246s 472ms/step - loss: 0.1303 - accuracy: 0.9704 - val_loss: 0.125

In [43]:
print(n)#19
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

19
Epoch 1/10
547/547 [==============================] - 261s 471ms/step - loss: 0.1842 - accuracy: 0.9564 - val_loss: 0.0392 - val_accuracy: 0.9908
Epoch 2/10
547/547 [==============================] - 258s 473ms/step - loss: 0.1362 - accuracy: 0.9659 - val_loss: 0.0299 - val_accuracy: 0.9944
Epoch 3/10
547/547 [==============================] - 260s 475ms/step - loss: 0.1261 - accuracy: 0.9687 - val_loss: 0.0237 - val_accuracy: 0.9946
Epoch 4/10
547/547 [==============================] - 259s 474ms/step - loss: 0.1245 - accuracy: 0.9694 - val_loss: 0.0399 - val_accuracy: 0.9916
Epoch 5/10
547/547 [==============================] - 258s 472ms/step - loss: 0.1227 - accuracy: 0.9695 - val_loss: 0.0355 - val_accuracy: 0.9934
Epoch 6/10
547/547 [==============================] - 257s 470ms/step - loss: 0.1236 - accuracy: 0.9698 - val_loss: 0.0434 - val_accuracy: 0.9936
Epoch 7/10
547/547 [==============================] - 259s 473ms/step - loss: 0.1412 - accuracy: 0.9684 - val_loss: 0.035

In [44]:
print(n)#last
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

20
Epoch 1/10
552/552 [==============================] - 264s 473ms/step - loss: 0.1955 - accuracy: 0.9542 - val_loss: 0.0478 - val_accuracy: 0.9880
Epoch 2/10
552/552 [==============================] - 260s 471ms/step - loss: 0.1402 - accuracy: 0.9652 - val_loss: 0.1120 - val_accuracy: 0.9796
Epoch 3/10
552/552 [==============================] - 260s 471ms/step - loss: 0.1292 - accuracy: 0.9664 - val_loss: 0.0310 - val_accuracy: 0.9936
Epoch 4/10
552/552 [==============================] - 263s 476ms/step - loss: 0.1292 - accuracy: 0.9686 - val_loss: 0.0294 - val_accuracy: 0.9936
Epoch 5/10
552/552 [==============================] - 260s 471ms/step - loss: 0.1282 - accuracy: 0.9681 - val_loss: 0.0955 - val_accuracy: 0.9842
Epoch 6/10
552/552 [==============================] - 267s 483ms/step - loss: 0.1333 - accuracy: 0.9680 - val_loss: 0.0283 - val_accuracy: 0.9952
Epoch 7/10
552/552 [==============================] - 276s 500ms/step - loss: 0.1274 - accuracy: 0.9715 - val_loss: 0.047

In [ ]:
len(models_lsa)

In [ ]:
models_lsa[-1].evaluate(x_val,y_val)

In [ ]:
len(models_lsa)

In [45]:
#new_model_lsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_lsa_e2"
new_model_lsa_dir  = "D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_lsa_e1"

i=0

for model in models_lsa:
    model.save(new_model_lsa_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_lsa_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_lsa_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_lsa_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_lsa_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_lsa_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_lsa_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_lsa_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_lsa_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gts

In [46]:
import gc

In [47]:
try:
    del lsa_values
    del top_images_by_lsa
    del top_labels_by_lsa
    del image_sets_lsa
    del label_sets_lsa
    del models_lsa
except:
    print("Error")

In [48]:
gc.collect()

595889

In [ ]:
loading = False

models_lsa = []

if loading:
    for i in range(10,20):
        model_lsa_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_lsa_e1_"+str(i)
        print(model_lsa_dir)
        model =utils.My_model('gtsrb',True,model_lsa_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_lsa.append(model)
       
    

## Training guided by DSA

In [49]:
m = 1750
n = 0
image_sets_dsa = []
label_sets_dsa = []


for i in range((len(top_images_by_dsa)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_dsa)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_dsa)%m))
        top_images_by_dsa_n = np.array(top_images_by_dsa[:n+m+(len(top_images_by_dsa)%m)])
        top_labels_by_dsa_n = np.array(top_labels_by_dsa[:n+m+(len(top_images_by_dsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_dsa_n = np.array(top_images_by_dsa[:n+m])
        top_labels_by_dsa_n = np.array(top_labels_by_dsa[:n+m])
    image_sets_dsa.append(top_images_by_dsa_n)
    label_sets_dsa.append(top_labels_by_dsa_n)
    print(len(top_images_by_dsa_n))
    n += m



0 :
0  ->  1750
1750
1 :
0  ->  3500
3500
2 :
0  ->  5250
5250
3 :
0  ->  7000
7000
4 :
0  ->  8750
8750
5 :
0  ->  10500
10500
6 :
0  ->  12250
12250
7 :
0  ->  14000
14000
8 :
0  ->  15750
15750
9 :
0  ->  17500
17500
10 :
0  ->  19250
19250
11 :
0  ->  21000
21000
12 :
0  ->  22750
22750
13 :
0  ->  24500
24500
14 :
0  ->  26250
26250
15 :
0  ->  28000
28000
16 :
0  ->  29750
29750
17 :
0  ->  31500
31500
18 :
0  ->  33250
33250
19 :
0  ->  35000
35000
20 :
Last
0  ->  37037
35287


In [50]:
print(model_dir)

models_dsa = []
for i in range(len(label_sets_dsa)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_dsa.append(model)


C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06/
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled
20 :
Model loaded correctly
Model compiled


In [51]:
n=0
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

0
Epoch 1/10
28/28 [==============================] - 24s 734ms/step - loss: 2.8782 - accuracy: 0.3874 - val_loss: 0.1395 - val_accuracy: 0.9801
Epoch 2/10
28/28 [==============================] - 20s 714ms/step - loss: 1.4450 - accuracy: 0.6159 - val_loss: 0.0683 - val_accuracy: 0.9796
Epoch 3/10
28/28 [==============================] - 20s 739ms/step - loss: 1.2367 - accuracy: 0.6523 - val_loss: 0.0664 - val_accuracy: 0.9811
Epoch 4/10
28/28 [==============================] - 22s 786ms/step - loss: 0.9845 - accuracy: 0.7238 - val_loss: 0.0577 - val_accuracy: 0.9862
Epoch 5/10
28/28 [==============================] - 21s 769ms/step - loss: 0.8484 - accuracy: 0.7499 - val_loss: 0.0636 - val_accuracy: 0.9816
Epoch 6/10
28/28 [==============================] - 21s 753ms/step - loss: 0.7421 - accuracy: 0.7663 - val_loss: 0.1593 - val_accuracy: 0.9610
Epoch 7/10
28/28 [==============================] - 21s 762ms/step - loss: 0.7199 - accuracy: 0.8106 - val_loss: 0.0722 - val_accuracy: 0.97

In [52]:
n=1
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

1
Epoch 1/10
55/55 [==============================] - 39s 646ms/step - loss: 1.2882 - accuracy: 0.6980 - val_loss: 0.0397 - val_accuracy: 0.9908
Epoch 2/10
55/55 [==============================] - 35s 648ms/step - loss: 0.7389 - accuracy: 0.8016 - val_loss: 0.0417 - val_accuracy: 0.9890
Epoch 3/10
55/55 [==============================] - 35s 646ms/step - loss: 0.6072 - accuracy: 0.8353 - val_loss: 0.0758 - val_accuracy: 0.9829
Epoch 4/10
55/55 [==============================] - 36s 649ms/step - loss: 0.4917 - accuracy: 0.8552 - val_loss: 0.0587 - val_accuracy: 0.9849
Epoch 5/10
55/55 [==============================] - 36s 655ms/step - loss: 0.4719 - accuracy: 0.8600 - val_loss: 0.0370 - val_accuracy: 0.9939
Epoch 6/10
55/55 [==============================] - 36s 649ms/step - loss: 0.3838 - accuracy: 0.8899 - val_loss: 0.0636 - val_accuracy: 0.9844
Epoch 7/10
55/55 [==============================] - 36s 650ms/step - loss: 0.3687 - accuracy: 0.8869 - val_loss: 0.0579 - val_accuracy: 0.98

In [53]:
n=2
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

2
Epoch 1/10
83/83 [==============================] - 54s 605ms/step - loss: 0.9661 - accuracy: 0.7716 - val_loss: 0.0507 - val_accuracy: 0.9870
Epoch 2/10
83/83 [==============================] - 50s 605ms/step - loss: 0.5532 - accuracy: 0.8513 - val_loss: 0.0445 - val_accuracy: 0.9872
Epoch 3/10
83/83 [==============================] - 50s 607ms/step - loss: 0.3728 - accuracy: 0.8926 - val_loss: 0.0502 - val_accuracy: 0.9900
Epoch 4/10
83/83 [==============================] - 50s 609ms/step - loss: 0.3756 - accuracy: 0.8934 - val_loss: 0.1486 - val_accuracy: 0.9604
Epoch 5/10
83/83 [==============================] - 50s 607ms/step - loss: 0.3449 - accuracy: 0.8978 - val_loss: 0.0565 - val_accuracy: 0.9852
Epoch 6/10
83/83 [==============================] - 51s 610ms/step - loss: 0.2856 - accuracy: 0.9169 - val_loss: 0.0427 - val_accuracy: 0.9916
Epoch 7/10
83/83 [==============================] - 51s 611ms/step - loss: 0.2541 - accuracy: 0.9304 - val_loss: 0.0401 - val_accuracy: 0.99

In [54]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

3
Epoch 1/10
110/110 [==============================] - 70s 603ms/step - loss: 0.6881 - accuracy: 0.8420 - val_loss: 0.0361 - val_accuracy: 0.9913
Epoch 2/10
110/110 [==============================] - 66s 604ms/step - loss: 0.3736 - accuracy: 0.8990 - val_loss: 0.0780 - val_accuracy: 0.9814
Epoch 3/10
110/110 [==============================] - 66s 603ms/step - loss: 0.3230 - accuracy: 0.9109 - val_loss: 0.0396 - val_accuracy: 0.9923
Epoch 4/10
110/110 [==============================] - 67s 605ms/step - loss: 0.2718 - accuracy: 0.9191 - val_loss: 0.0593 - val_accuracy: 0.9865
Epoch 5/10
110/110 [==============================] - 67s 607ms/step - loss: 0.2507 - accuracy: 0.9278 - val_loss: 0.1209 - val_accuracy: 0.9717
Epoch 6/10
110/110 [==============================] - 70s 633ms/step - loss: 0.2251 - accuracy: 0.9357 - val_loss: 0.0501 - val_accuracy: 0.9903
Epoch 7/10
110/110 [==============================] - 66s 604ms/step - loss: 0.1840 - accuracy: 0.9471 - val_loss: 0.0412 - val_

In [55]:
print(n)#4

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

4
Epoch 1/10
137/137 [==============================] - 85s 598ms/step - loss: 0.5614 - accuracy: 0.8771 - val_loss: 0.0421 - val_accuracy: 0.9918
Epoch 2/10
137/137 [==============================] - 81s 593ms/step - loss: 0.3077 - accuracy: 0.9184 - val_loss: 0.0400 - val_accuracy: 0.9906
Epoch 3/10
137/137 [==============================] - 81s 590ms/step - loss: 0.2671 - accuracy: 0.9235 - val_loss: 0.0563 - val_accuracy: 0.9893
Epoch 4/10
137/137 [==============================] - 81s 591ms/step - loss: 0.2143 - accuracy: 0.9394 - val_loss: 0.0527 - val_accuracy: 0.9903
Epoch 5/10
137/137 [==============================] - 81s 590ms/step - loss: 0.2174 - accuracy: 0.9416 - val_loss: 0.0503 - val_accuracy: 0.9900
Epoch 6/10
137/137 [==============================] - 81s 591ms/step - loss: 0.1925 - accuracy: 0.9454 - val_loss: 0.0465 - val_accuracy: 0.9883
Epoch 7/10
137/137 [==============================] - 81s 592ms/step - loss: 0.1704 - accuracy: 0.9510 - val_loss: 0.0788 - val_

In [56]:
n=5
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

5
Epoch 1/10
165/165 [==============================] - 99s 581ms/step - loss: 0.4494 - accuracy: 0.8948 - val_loss: 0.0398 - val_accuracy: 0.9918
Epoch 2/10
165/165 [==============================] - 98s 597ms/step - loss: 0.2796 - accuracy: 0.9258 - val_loss: 0.0455 - val_accuracy: 0.9903
Epoch 3/10
165/165 [==============================] - 98s 595ms/step - loss: 0.2355 - accuracy: 0.9354 - val_loss: 0.0542 - val_accuracy: 0.9852
Epoch 4/10
165/165 [==============================] - 98s 593ms/step - loss: 0.2248 - accuracy: 0.9422 - val_loss: 0.0589 - val_accuracy: 0.9870
Epoch 5/10
165/165 [==============================] - 98s 591ms/step - loss: 0.1864 - accuracy: 0.9457 - val_loss: 0.0474 - val_accuracy: 0.9888
Epoch 6/10
165/165 [==============================] - 98s 591ms/step - loss: 0.1666 - accuracy: 0.9542 - val_loss: 0.0715 - val_accuracy: 0.9821
Epoch 7/10
165/165 [==============================] - 98s 592ms/step - loss: 0.1932 - accuracy: 0.9514 - val_loss: 0.0526 - val_

In [57]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

6
Epoch 1/10
192/192 [==============================] - 119s 601ms/step - loss: 0.4380 - accuracy: 0.8992 - val_loss: 0.0346 - val_accuracy: 0.9918
Epoch 2/10
192/192 [==============================] - 118s 615ms/step - loss: 0.2271 - accuracy: 0.9370 - val_loss: 0.0486 - val_accuracy: 0.9883
Epoch 3/10
192/192 [==============================] - 117s 608ms/step - loss: 0.2137 - accuracy: 0.9405 - val_loss: 0.0497 - val_accuracy: 0.9878
Epoch 4/10
192/192 [==============================] - 117s 609ms/step - loss: 0.1763 - accuracy: 0.9494 - val_loss: 0.0681 - val_accuracy: 0.9870
Epoch 5/10
192/192 [==============================] - 117s 610ms/step - loss: 0.1927 - accuracy: 0.9487 - val_loss: 0.0557 - val_accuracy: 0.9872
Epoch 6/10
192/192 [==============================] - 117s 611ms/step - loss: 0.1495 - accuracy: 0.9600 - val_loss: 0.0777 - val_accuracy: 0.9829
Epoch 7/10
192/192 [==============================] - 117s 609ms/step - loss: 0.1397 - accuracy: 0.9650 - val_loss: 0.0807

In [58]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

7
Epoch 1/10
219/219 [==============================] - 136s 604ms/step - loss: 0.3528 - accuracy: 0.9159 - val_loss: 0.0630 - val_accuracy: 0.9880
Epoch 2/10
219/219 [==============================] - 129s 588ms/step - loss: 0.2234 - accuracy: 0.9422 - val_loss: 0.0454 - val_accuracy: 0.9898
Epoch 3/10
219/219 [==============================] - 116s 531ms/step - loss: 0.1758 - accuracy: 0.9526 - val_loss: 0.0453 - val_accuracy: 0.9872
Epoch 4/10
219/219 [==============================] - 116s 530ms/step - loss: 0.1733 - accuracy: 0.9550 - val_loss: 0.0637 - val_accuracy: 0.9883
Epoch 5/10
219/219 [==============================] - 116s 531ms/step - loss: 0.1768 - accuracy: 0.9540 - val_loss: 0.0652 - val_accuracy: 0.9855
Epoch 6/10
219/219 [==============================] - 118s 538ms/step - loss: 0.1553 - accuracy: 0.9570 - val_loss: 0.0403 - val_accuracy: 0.9913
Epoch 7/10
219/219 [==============================] - 117s 533ms/step - loss: 0.1313 - accuracy: 0.9635 - val_loss: 0.0355

In [59]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

8
Epoch 1/10
247/247 [==============================] - 135s 533ms/step - loss: 0.3178 - accuracy: 0.9231 - val_loss: 0.0303 - val_accuracy: 0.9923
Epoch 2/10
247/247 [==============================] - 132s 533ms/step - loss: 0.2022 - accuracy: 0.9464 - val_loss: 0.0501 - val_accuracy: 0.9885
Epoch 3/10
247/247 [==============================] - 132s 536ms/step - loss: 0.1811 - accuracy: 0.9519 - val_loss: 0.0383 - val_accuracy: 0.9918
Epoch 4/10
247/247 [==============================] - 153s 620ms/step - loss: 0.1665 - accuracy: 0.9573 - val_loss: 0.4071 - val_accuracy: 0.9140
Epoch 5/10
247/247 [==============================] - 189s 765ms/step - loss: 0.1670 - accuracy: 0.9549 - val_loss: 0.0340 - val_accuracy: 0.9913
Epoch 6/10
247/247 [==============================] - 194s 784ms/step - loss: 0.1458 - accuracy: 0.9620 - val_loss: 0.0667 - val_accuracy: 0.9821
Epoch 7/10
247/247 [==============================] - 192s 778ms/step - loss: 0.1594 - accuracy: 0.9605 - val_loss: 0.0489

In [60]:
print(n)#9

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

9
Epoch 1/10
274/274 [==============================] - 214s 741ms/step - loss: 0.3052 - accuracy: 0.9272 - val_loss: 0.0551 - val_accuracy: 0.9880
Epoch 2/10
274/274 [==============================] - 184s 671ms/step - loss: 0.1957 - accuracy: 0.9490 - val_loss: 0.0274 - val_accuracy: 0.9934
Epoch 3/10
274/274 [==============================] - 188s 685ms/step - loss: 0.1722 - accuracy: 0.9542 - val_loss: 0.0380 - val_accuracy: 0.9893
Epoch 4/10
274/274 [==============================] - 188s 685ms/step - loss: 0.1670 - accuracy: 0.9565 - val_loss: 0.0365 - val_accuracy: 0.9895
Epoch 5/10
274/274 [==============================] - 191s 698ms/step - loss: 0.1505 - accuracy: 0.9598 - val_loss: 0.0438 - val_accuracy: 0.9913
Epoch 6/10
274/274 [==============================] - 188s 685ms/step - loss: 0.1450 - accuracy: 0.9635 - val_loss: 0.0405 - val_accuracy: 0.9911
Epoch 7/10
274/274 [==============================] - 182s 664ms/step - loss: 0.1379 - accuracy: 0.9639 - val_loss: 0.0741

In [61]:
n=10
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

10
Epoch 1/10
301/301 [==============================] - 167s 544ms/step - loss: 0.2724 - accuracy: 0.9345 - val_loss: 0.0431 - val_accuracy: 0.9898
Epoch 2/10
301/301 [==============================] - 170s 566ms/step - loss: 0.2048 - accuracy: 0.9462 - val_loss: 0.0409 - val_accuracy: 0.9895
Epoch 3/10
301/301 [==============================] - 163s 542ms/step - loss: 0.1603 - accuracy: 0.9580 - val_loss: 0.0449 - val_accuracy: 0.9903
Epoch 4/10
301/301 [==============================] - 166s 552ms/step - loss: 0.1591 - accuracy: 0.9580 - val_loss: 0.0422 - val_accuracy: 0.9906
Epoch 5/10
301/301 [==============================] - 163s 542ms/step - loss: 0.1542 - accuracy: 0.9605 - val_loss: 0.0405 - val_accuracy: 0.9936
Epoch 6/10
301/301 [==============================] - 164s 544ms/step - loss: 0.1485 - accuracy: 0.9622 - val_loss: 0.0509 - val_accuracy: 0.9911
Epoch 7/10
301/301 [==============================] - 164s 544ms/step - loss: 0.1461 - accuracy: 0.9636 - val_loss: 0.032

In [62]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

11
Epoch 1/10
329/329 [==============================] - 209s 626ms/step - loss: 0.2741 - accuracy: 0.9367 - val_loss: 0.0366 - val_accuracy: 0.9913
Epoch 2/10
329/329 [==============================] - 207s 630ms/step - loss: 0.1784 - accuracy: 0.9532 - val_loss: 0.0283 - val_accuracy: 0.9934
Epoch 3/10
329/329 [==============================] - 207s 630ms/step - loss: 0.1698 - accuracy: 0.9561 - val_loss: 0.0367 - val_accuracy: 0.9908
Epoch 4/10
329/329 [==============================] - 207s 630ms/step - loss: 0.1503 - accuracy: 0.9620 - val_loss: 0.0942 - val_accuracy: 0.9804
Epoch 5/10
329/329 [==============================] - 209s 636ms/step - loss: 0.1488 - accuracy: 0.9605 - val_loss: 0.0487 - val_accuracy: 0.9885
Epoch 6/10
329/329 [==============================] - 209s 635ms/step - loss: 0.1317 - accuracy: 0.9650 - val_loss: 0.0546 - val_accuracy: 0.9913
Epoch 7/10
329/329 [==============================] - 208s 633ms/step - loss: 0.1370 - accuracy: 0.9670 - val_loss: 0.072

In [63]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

12
Epoch 1/10
356/356 [==============================] - 218s 601ms/step - loss: 0.2785 - accuracy: 0.9360 - val_loss: 0.0672 - val_accuracy: 0.9916
Epoch 2/10
356/356 [==============================] - 203s 569ms/step - loss: 0.1732 - accuracy: 0.9561 - val_loss: 0.0352 - val_accuracy: 0.9926
Epoch 3/10
356/356 [==============================] - 203s 570ms/step - loss: 0.1437 - accuracy: 0.9614 - val_loss: 0.1065 - val_accuracy: 0.9730
Epoch 4/10
356/356 [==============================] - 202s 568ms/step - loss: 0.1512 - accuracy: 0.9602 - val_loss: 0.0232 - val_accuracy: 0.9954
Epoch 5/10
356/356 [==============================] - 202s 567ms/step - loss: 0.1466 - accuracy: 0.9638 - val_loss: 0.0997 - val_accuracy: 0.9775
Epoch 6/10
356/356 [==============================] - 201s 564ms/step - loss: 0.1451 - accuracy: 0.9639 - val_loss: 0.0505 - val_accuracy: 0.9890
Epoch 7/10
356/356 [==============================] - 201s 564ms/step - loss: 0.1432 - accuracy: 0.9656 - val_loss: 0.032

In [64]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

13
Epoch 1/10
383/383 [==============================] - 255s 656ms/step - loss: 0.2571 - accuracy: 0.9430 - val_loss: 0.0448 - val_accuracy: 0.9911
Epoch 2/10
383/383 [==============================] - 252s 659ms/step - loss: 0.1642 - accuracy: 0.9559 - val_loss: 0.0376 - val_accuracy: 0.9911
Epoch 3/10
383/383 [==============================] - 389s 1s/step - loss: 0.1518 - accuracy: 0.9592 - val_loss: 0.0544 - val_accuracy: 0.9885
Epoch 4/10
383/383 [==============================] - 540s 1s/step - loss: 0.1600 - accuracy: 0.9593 - val_loss: 0.0259 - val_accuracy: 0.9939
Epoch 5/10
383/383 [==============================] - 532s 1s/step - loss: 0.1406 - accuracy: 0.9647 - val_loss: 0.0878 - val_accuracy: 0.9880
Epoch 6/10
383/383 [==============================] - 544s 1s/step - loss: 0.1317 - accuracy: 0.9676 - val_loss: 0.0523 - val_accuracy: 0.9880
Epoch 7/10
383/383 [==============================] - 501s 1s/step - loss: 0.1440 - accuracy: 0.9674 - val_loss: 0.0344 - val_accurac

In [65]:
print(n)#14

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

14
Epoch 1/10
411/411 [==============================] - 572s 1s/step - loss: 0.2163 - accuracy: 0.9477 - val_loss: 0.0330 - val_accuracy: 0.9926
Epoch 2/10
411/411 [==============================] - 569s 1s/step - loss: 0.1656 - accuracy: 0.9584 - val_loss: 0.0351 - val_accuracy: 0.9903
Epoch 3/10
411/411 [==============================] - 565s 1s/step - loss: 0.1493 - accuracy: 0.9623 - val_loss: 0.0394 - val_accuracy: 0.9921
Epoch 4/10
411/411 [==============================] - 490s 1s/step - loss: 0.1394 - accuracy: 0.9642 - val_loss: 0.0651 - val_accuracy: 0.9906
Epoch 5/10
411/411 [==============================] - 437s 1s/step - loss: 0.1441 - accuracy: 0.9629 - val_loss: 0.0544 - val_accuracy: 0.9862
Epoch 6/10
411/411 [==============================] - 441s 1s/step - loss: 0.1371 - accuracy: 0.9668 - val_loss: 0.0663 - val_accuracy: 0.9855
Epoch 7/10
411/411 [==============================] - 474s 1s/step - loss: 0.1400 - accuracy: 0.9672 - val_loss: 0.0615 - val_accuracy: 0.9

In [66]:
n=15
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

15
Epoch 1/10
438/438 [==============================] - 580s 1s/step - loss: 0.2175 - accuracy: 0.9498 - val_loss: 0.0331 - val_accuracy: 0.9939
Epoch 2/10
438/438 [==============================] - 574s 1s/step - loss: 0.1563 - accuracy: 0.9602 - val_loss: 0.0461 - val_accuracy: 0.9913
Epoch 3/10
438/438 [==============================] - 573s 1s/step - loss: 0.1407 - accuracy: 0.9650 - val_loss: 0.0285 - val_accuracy: 0.9929
Epoch 4/10
438/438 [==============================] - 541s 1s/step - loss: 0.1394 - accuracy: 0.9654 - val_loss: 0.0296 - val_accuracy: 0.9936
Epoch 5/10
438/438 [==============================] - 471s 1s/step - loss: 0.1322 - accuracy: 0.9679 - val_loss: 0.0535 - val_accuracy: 0.9913
Epoch 6/10
438/438 [==============================] - 472s 1s/step - loss: 0.1345 - accuracy: 0.9659 - val_loss: 0.1645 - val_accuracy: 0.9694
Epoch 7/10
438/438 [==============================] - 466s 1s/step - loss: 0.1317 - accuracy: 0.9683 - val_loss: 0.0581 - val_accuracy: 0.9

In [67]:
import time
time.sleep(400)

In [68]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

16
Epoch 1/10
465/465 [==============================] - 586s 1s/step - loss: 0.2166 - accuracy: 0.9484 - val_loss: 0.0277 - val_accuracy: 0.9946
Epoch 2/10
465/465 [==============================] - 501s 1s/step - loss: 0.1482 - accuracy: 0.9609 - val_loss: 0.0296 - val_accuracy: 0.9934
Epoch 3/10
465/465 [==============================] - 500s 1s/step - loss: 0.1461 - accuracy: 0.9636 - val_loss: 0.0361 - val_accuracy: 0.9921
Epoch 4/10
465/465 [==============================] - 528s 1s/step - loss: 0.1285 - accuracy: 0.9676 - val_loss: 0.0369 - val_accuracy: 0.9923
Epoch 5/10
465/465 [==============================] - 608s 1s/step - loss: 0.1427 - accuracy: 0.9655 - val_loss: 0.0281 - val_accuracy: 0.9941
Epoch 6/10
465/465 [==============================] - 610s 1s/step - loss: 0.1463 - accuracy: 0.9640 - val_loss: 0.0470 - val_accuracy: 0.9898
Epoch 7/10
465/465 [==============================] - 603s 1s/step - loss: 0.1395 - accuracy: 0.9679 - val_loss: 0.0236 - val_accuracy: 0.9

In [ ]:

time.sleep(400)

In [69]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

17
Epoch 1/10
493/493 [==============================] - 543s 1s/step - loss: 0.1947 - accuracy: 0.9544 - val_loss: 0.0516 - val_accuracy: 0.9923
Epoch 2/10
493/493 [==============================] - 532s 1s/step - loss: 0.1489 - accuracy: 0.9634 - val_loss: 0.0268 - val_accuracy: 0.9934
Epoch 3/10
493/493 [==============================] - 620s 1s/step - loss: 0.1432 - accuracy: 0.9651 - val_loss: 0.0311 - val_accuracy: 0.9941
Epoch 4/10
493/493 [==============================] - 631s 1s/step - loss: 0.1284 - accuracy: 0.9675 - val_loss: 0.0395 - val_accuracy: 0.9888
Epoch 5/10
493/493 [==============================] - 638s 1s/step - loss: 0.1213 - accuracy: 0.9702 - val_loss: 0.0312 - val_accuracy: 0.9952
Epoch 6/10
493/493 [==============================] - 634s 1s/step - loss: 0.1375 - accuracy: 0.9680 - val_loss: 0.0225 - val_accuracy: 0.9944
Epoch 7/10
493/493 [==============================] - 639s 1s/step - loss: 0.1359 - accuracy: 0.9685 - val_loss: 0.0375 - val_accuracy: 0.9

In [70]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

18
Epoch 1/10
520/520 [==============================] - 671s 1s/step - loss: 0.1917 - accuracy: 0.9566 - val_loss: 0.0261 - val_accuracy: 0.9931
Epoch 2/10
520/520 [==============================] - 669s 1s/step - loss: 0.1405 - accuracy: 0.9631 - val_loss: 0.0971 - val_accuracy: 0.9778
Epoch 3/10
520/520 [==============================] - 665s 1s/step - loss: 0.1397 - accuracy: 0.9656 - val_loss: 0.0246 - val_accuracy: 0.9954
Epoch 4/10
520/520 [==============================] - 665s 1s/step - loss: 0.1307 - accuracy: 0.9682 - val_loss: 0.0598 - val_accuracy: 0.9860
Epoch 5/10
520/520 [==============================] - 667s 1s/step - loss: 0.1277 - accuracy: 0.9695 - val_loss: 0.0232 - val_accuracy: 0.9972
Epoch 6/10
520/520 [==============================] - 666s 1s/step - loss: 0.1218 - accuracy: 0.9702 - val_loss: 0.0323 - val_accuracy: 0.9941
Epoch 7/10
520/520 [==============================] - 668s 1s/step - loss: 0.1302 - accuracy: 0.9691 - val_loss: 0.0621 - val_accuracy: 0.9

In [71]:
n=19
print(n)#19

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

19
Epoch 1/10
547/547 [==============================] - 631s 1s/step - loss: 0.1914 - accuracy: 0.9556 - val_loss: 0.0350 - val_accuracy: 0.9939
Epoch 2/10
547/547 [==============================] - 641s 1s/step - loss: 0.1452 - accuracy: 0.9637 - val_loss: 0.0555 - val_accuracy: 0.9918
Epoch 3/10
547/547 [==============================] - 704s 1s/step - loss: 0.1320 - accuracy: 0.9678 - val_loss: 0.0223 - val_accuracy: 0.9946
Epoch 4/10
547/547 [==============================] - 708s 1s/step - loss: 0.1339 - accuracy: 0.9679 - val_loss: 0.0275 - val_accuracy: 0.9939
Epoch 5/10
547/547 [==============================] - 701s 1s/step - loss: 0.1400 - accuracy: 0.9687 - val_loss: 0.0546 - val_accuracy: 0.9941
Epoch 6/10
547/547 [==============================] - 701s 1s/step - loss: 0.1339 - accuracy: 0.9697 - val_loss: 0.0220 - val_accuracy: 0.9949
Epoch 7/10
547/547 [==============================] - 692s 1s/step - loss: 0.1301 - accuracy: 0.9706 - val_loss: 0.0329 - val_accuracy: 0.9

In [72]:
n=20
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

20
Epoch 1/10
552/552 [==============================] - 653s 1s/step - loss: 0.1881 - accuracy: 0.9549 - val_loss: 0.0221 - val_accuracy: 0.9954
Epoch 2/10
552/552 [==============================] - 619s 1s/step - loss: 0.1464 - accuracy: 0.9634 - val_loss: 0.0380 - val_accuracy: 0.9946
Epoch 3/10
552/552 [==============================] - 636s 1s/step - loss: 0.1285 - accuracy: 0.9692 - val_loss: 0.0311 - val_accuracy: 0.9913
Epoch 4/10
552/552 [==============================] - 707s 1s/step - loss: 0.1301 - accuracy: 0.9691 - val_loss: 0.0354 - val_accuracy: 0.9944
Epoch 5/10
552/552 [==============================] - 698s 1s/step - loss: 0.1345 - accuracy: 0.9697 - val_loss: 0.0392 - val_accuracy: 0.9944
Epoch 6/10
552/552 [==============================] - 708s 1s/step - loss: 0.1378 - accuracy: 0.9671 - val_loss: 0.0266 - val_accuracy: 0.9918
Epoch 7/10
552/552 [==============================] - 704s 1s/step - loss: 0.1332 - accuracy: 0.9688 - val_loss: 0.0396 - val_accuracy: 0.9

In [73]:
new_model_dsa_dir  = "D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_dsa_e1"

i=0

for model in models_dsa:
    model.save(new_model_dsa_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_dsa_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_dsa_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_dsa_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_dsa_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_dsa_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_dsa_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_dsa_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_dsa_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gts

In [ ]:
loading = True

models_dsa = []

if loading:
    for i in range(20):
        model_dsa_dir = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_dsa_e2_"+str(i)
        print(model_dsa_dir)
        model =utils.My_model('gtsrb',True,model_dsa_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_dsa.append(model)
       
    

In [74]:
try:
    del dsa_values
    del top_images_by_dsa
    del top_labels_by_dsa
    del image_sets_dsa
    del label_sets_dsa
    del models_dsa
except:
    print("Error")

In [75]:
gc.collect()

559646

## Uncertainty

In [18]:
dg_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/deep_gini_values.npy"

deep_gini_values = np.load(dg_direction)


In [19]:
# Obtaining top n images by dg values
top_images_by_dg  = utils.get_x_of_indexes(list(np.flip(np.argsort(deep_gini_values))),x_train_and_adversary)
top_labels_by_dg = utils.get_x_of_indexes(list(np.flip(np.argsort(deep_gini_values))),y_train_and_adversary)

In [20]:
m = n_data_points
n = 0
image_sets_dg = []
label_sets_dg = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_dg)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_dg)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_dg)%m))
        top_images_by_dg_n = np.array(top_images_by_dg[:n+m+(len(top_images_by_dg)%m)])
        top_labels_by_dg_n = np.array(top_labels_by_dg[:n+m+(len(top_images_by_dg)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_dg_n = np.array(top_images_by_dg[:n+m])
        top_labels_by_dg_n = np.array(top_labels_by_dg[:n+m])
    image_sets_dg.append(top_images_by_dg_n)
    label_sets_dg.append(top_labels_by_dg_n)
    print(len(top_images_by_dg_n))
    n += m



0 :
0  ->  1750
1750
1 :
0  ->  3500
3500
2 :
0  ->  5250
5250
3 :
0  ->  7000
7000
4 :
0  ->  8750
8750
5 :
0  ->  10500
10500
6 :
0  ->  12250
12250
7 :
0  ->  14000
14000
8 :
0  ->  15750
15750
9 :
0  ->  17500
17500
10 :
0  ->  19250
19250
11 :
0  ->  21000
21000
12 :
0  ->  22750
22750
13 :
0  ->  24500
24500
14 :
0  ->  26250
26250
15 :
0  ->  28000
28000
16 :
0  ->  29750
29750
17 :
0  ->  31500
31500
18 :
0  ->  33250
33250
19 :
0  ->  35000
35000
20 :
Last
0  ->  37037
35287


In [21]:
print(model_dir)
models_dg = []
#for i in range(len(label_sets_lsa)):
for i in range(len(label_sets_dg)):
    print(i,":")
    model = utils.My_model(dataset,True, model_dir)
    model.compile_model()
    models_dg.append(model)

C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06/
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled
20 :
Model loaded correctly
Model compiled


In [80]:
n=0
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

0
Epoch 1/10
28/28 [==============================] - 42s 1s/step - loss: 2.6309 - accuracy: 0.3419 - val_loss: 0.0367 - val_accuracy: 0.9911
Epoch 2/10
28/28 [==============================] - 37s 1s/step - loss: 1.5975 - accuracy: 0.5593 - val_loss: 0.0351 - val_accuracy: 0.9903
Epoch 3/10
28/28 [==============================] - 35s 1s/step - loss: 1.2723 - accuracy: 0.6408 - val_loss: 0.0393 - val_accuracy: 0.9885
Epoch 4/10
28/28 [==============================] - 37s 1s/step - loss: 1.0829 - accuracy: 0.6889 - val_loss: 0.0805 - val_accuracy: 0.9778
Epoch 5/10
28/28 [==============================] - 38s 1s/step - loss: 0.9609 - accuracy: 0.7256 - val_loss: 0.0292 - val_accuracy: 0.9908
Epoch 6/10
28/28 [==============================] - 36s 1s/step - loss: 0.7594 - accuracy: 0.7672 - val_loss: 0.0418 - val_accuracy: 0.9885
Epoch 7/10
28/28 [==============================] - 37s 1s/step - loss: 0.6335 - accuracy: 0.8192 - val_loss: 0.0960 - val_accuracy: 0.9791
Epoch 8/10
28/28 [

In [81]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

1
Epoch 1/10
55/55 [==============================] - 81s 1s/step - loss: 1.6669 - accuracy: 0.6046 - val_loss: 0.0239 - val_accuracy: 0.9934
Epoch 2/10
55/55 [==============================] - 84s 2s/step - loss: 0.9181 - accuracy: 0.7620 - val_loss: 0.0208 - val_accuracy: 0.9954
Epoch 3/10
55/55 [==============================] - 85s 2s/step - loss: 0.6963 - accuracy: 0.8108 - val_loss: 0.0158 - val_accuracy: 0.9959
Epoch 4/10
55/55 [==============================] - 86s 2s/step - loss: 0.5937 - accuracy: 0.8184 - val_loss: 0.0155 - val_accuracy: 0.9954
Epoch 5/10
55/55 [==============================] - 85s 2s/step - loss: 0.5689 - accuracy: 0.8343 - val_loss: 0.0143 - val_accuracy: 0.9974
Epoch 6/10
55/55 [==============================] - 84s 2s/step - loss: 0.4895 - accuracy: 0.8542 - val_loss: 0.0886 - val_accuracy: 0.9768
Epoch 7/10
55/55 [==============================] - 83s 2s/step - loss: 0.4812 - accuracy: 0.8613 - val_loss: 0.0397 - val_accuracy: 0.9918
Epoch 8/10
55/55 [

In [82]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

2
Epoch 1/10
83/83 [==============================] - 121s 1s/step - loss: 0.9875 - accuracy: 0.7672 - val_loss: 0.0150 - val_accuracy: 0.9959
Epoch 2/10
83/83 [==============================] - 125s 2s/step - loss: 0.6338 - accuracy: 0.8222 - val_loss: 0.0322 - val_accuracy: 0.9944
Epoch 3/10
83/83 [==============================] - 126s 2s/step - loss: 0.5355 - accuracy: 0.8502 - val_loss: 0.0191 - val_accuracy: 0.9957
Epoch 4/10
83/83 [==============================] - 125s 2s/step - loss: 0.4442 - accuracy: 0.8694 - val_loss: 0.0373 - val_accuracy: 0.9939
Epoch 5/10
83/83 [==============================] - 126s 2s/step - loss: 0.3973 - accuracy: 0.8899 - val_loss: 0.0194 - val_accuracy: 0.9962
Epoch 6/10
83/83 [==============================] - 126s 2s/step - loss: 0.3594 - accuracy: 0.9001 - val_loss: 0.0236 - val_accuracy: 0.9954
Epoch 7/10
83/83 [==============================] - 125s 2s/step - loss: 0.3335 - accuracy: 0.9051 - val_loss: 0.0198 - val_accuracy: 0.9964
Epoch 8/10


In [83]:
n=3
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

3
Epoch 1/10
110/110 [==============================] - 145s 1s/step - loss: 0.7515 - accuracy: 0.8166 - val_loss: 0.0962 - val_accuracy: 0.9903
Epoch 2/10
110/110 [==============================] - 161s 1s/step - loss: 0.4990 - accuracy: 0.8635 - val_loss: 0.0329 - val_accuracy: 0.9926
Epoch 3/10
110/110 [==============================] - 168s 2s/step - loss: 0.4054 - accuracy: 0.8846 - val_loss: 0.0186 - val_accuracy: 0.9959
Epoch 4/10
110/110 [==============================] - 173s 2s/step - loss: 0.3303 - accuracy: 0.9112 - val_loss: 0.0268 - val_accuracy: 0.9959
Epoch 5/10
110/110 [==============================] - 167s 2s/step - loss: 0.2948 - accuracy: 0.9180 - val_loss: 0.0274 - val_accuracy: 0.9954
Epoch 6/10
110/110 [==============================] - 166s 2s/step - loss: 0.3040 - accuracy: 0.9151 - val_loss: 0.0172 - val_accuracy: 0.9967
Epoch 7/10
110/110 [==============================] - 149s 1s/step - loss: 0.2797 - accuracy: 0.9246 - val_loss: 0.0210 - val_accuracy: 0.99

In [84]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

4
Epoch 1/10
137/137 [==============================] - 219s 2s/step - loss: 0.6536 - accuracy: 0.8441 - val_loss: 0.0196 - val_accuracy: 0.9952
Epoch 2/10
137/137 [==============================] - 216s 2s/step - loss: 0.3812 - accuracy: 0.8956 - val_loss: 0.0127 - val_accuracy: 0.9964
Epoch 3/10
137/137 [==============================] - 214s 2s/step - loss: 0.3303 - accuracy: 0.9089 - val_loss: 0.0170 - val_accuracy: 0.9959
Epoch 4/10
137/137 [==============================] - 214s 2s/step - loss: 0.3081 - accuracy: 0.9138 - val_loss: 0.0175 - val_accuracy: 0.9974
Epoch 5/10
137/137 [==============================] - 213s 2s/step - loss: 0.2588 - accuracy: 0.9302 - val_loss: 0.0237 - val_accuracy: 0.9946
Epoch 6/10
137/137 [==============================] - 211s 2s/step - loss: 0.2482 - accuracy: 0.9316 - val_loss: 0.0257 - val_accuracy: 0.9939
Epoch 7/10
137/137 [==============================] - 215s 2s/step - loss: 0.2325 - accuracy: 0.9352 - val_loss: 0.0417 - val_accuracy: 0.99

In [85]:
print(n)#5
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

5
Epoch 1/10
165/165 [==============================] - 249s 1s/step - loss: 0.5562 - accuracy: 0.8744 - val_loss: 0.0180 - val_accuracy: 0.9959
Epoch 2/10
165/165 [==============================] - 236s 1s/step - loss: 0.3457 - accuracy: 0.9123 - val_loss: 0.0261 - val_accuracy: 0.9939
Epoch 3/10
165/165 [==============================] - 251s 2s/step - loss: 0.2887 - accuracy: 0.9224 - val_loss: 0.0137 - val_accuracy: 0.9974
Epoch 4/10
165/165 [==============================] - 234s 1s/step - loss: 0.2548 - accuracy: 0.9320 - val_loss: 0.0165 - val_accuracy: 0.9959
Epoch 5/10
165/165 [==============================] - 243s 1s/step - loss: 0.2318 - accuracy: 0.9404 - val_loss: 0.0171 - val_accuracy: 0.9972
Epoch 6/10
165/165 [==============================] - 250s 2s/step - loss: 0.2262 - accuracy: 0.9382 - val_loss: 0.0191 - val_accuracy: 0.9952
Epoch 7/10
165/165 [==============================] - 254s 2s/step - loss: 0.2135 - accuracy: 0.9410 - val_loss: 0.0133 - val_accuracy: 0.99

In [86]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

6
Epoch 1/10
192/192 [==============================] - 267s 1s/step - loss: 0.4668 - accuracy: 0.8923 - val_loss: 0.0235 - val_accuracy: 0.9946
Epoch 2/10
192/192 [==============================] - 278s 1s/step - loss: 0.2991 - accuracy: 0.9195 - val_loss: 0.0294 - val_accuracy: 0.9944
Epoch 3/10
192/192 [==============================] - 276s 1s/step - loss: 0.2532 - accuracy: 0.9305 - val_loss: 0.0155 - val_accuracy: 0.9959
Epoch 4/10
192/192 [==============================] - 275s 1s/step - loss: 0.2252 - accuracy: 0.9349 - val_loss: 0.0162 - val_accuracy: 0.9962
Epoch 5/10
192/192 [==============================] - 274s 1s/step - loss: 0.2080 - accuracy: 0.9427 - val_loss: 0.0276 - val_accuracy: 0.9959
Epoch 6/10
192/192 [==============================] - 276s 1s/step - loss: 0.2069 - accuracy: 0.9464 - val_loss: 0.0317 - val_accuracy: 0.9916
Epoch 7/10
192/192 [==============================] - 279s 1s/step - loss: 0.2026 - accuracy: 0.9437 - val_loss: 0.0201 - val_accuracy: 0.99

In [87]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

7
Epoch 1/10
219/219 [==============================] - 311s 1s/step - loss: 0.4043 - accuracy: 0.9052 - val_loss: 0.0242 - val_accuracy: 0.9939
Epoch 2/10
219/219 [==============================] - 296s 1s/step - loss: 0.2729 - accuracy: 0.9260 - val_loss: 0.0339 - val_accuracy: 0.9957
Epoch 3/10
219/219 [==============================] - 304s 1s/step - loss: 0.2284 - accuracy: 0.9377 - val_loss: 0.0193 - val_accuracy: 0.9957
Epoch 4/10
219/219 [==============================] - 308s 1s/step - loss: 0.2084 - accuracy: 0.9428 - val_loss: 0.0168 - val_accuracy: 0.9967
Epoch 5/10
219/219 [==============================] - 315s 1s/step - loss: 0.1888 - accuracy: 0.9471 - val_loss: 0.0225 - val_accuracy: 0.9959
Epoch 6/10
219/219 [==============================] - 299s 1s/step - loss: 0.1786 - accuracy: 0.9511 - val_loss: 0.0277 - val_accuracy: 0.9941
Epoch 7/10
219/219 [==============================] - 311s 1s/step - loss: 0.1729 - accuracy: 0.9543 - val_loss: 0.0320 - val_accuracy: 0.99

In [88]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

8
Epoch 1/10
247/247 [==============================] - 341s 1s/step - loss: 0.3719 - accuracy: 0.9109 - val_loss: 0.0436 - val_accuracy: 0.9913
Epoch 2/10
247/247 [==============================] - 334s 1s/step - loss: 0.2513 - accuracy: 0.9356 - val_loss: 0.0218 - val_accuracy: 0.9944
Epoch 3/10
247/247 [==============================] - 337s 1s/step - loss: 0.2192 - accuracy: 0.9432 - val_loss: 0.0363 - val_accuracy: 0.9939
Epoch 4/10
247/247 [==============================] - 336s 1s/step - loss: 0.1941 - accuracy: 0.9445 - val_loss: 0.0356 - val_accuracy: 0.9926
Epoch 5/10
247/247 [==============================] - 334s 1s/step - loss: 0.1748 - accuracy: 0.9520 - val_loss: 0.0338 - val_accuracy: 0.9967
Epoch 6/10
247/247 [==============================] - 344s 1s/step - loss: 0.1860 - accuracy: 0.9492 - val_loss: 0.1594 - val_accuracy: 0.9645
Epoch 7/10
247/247 [==============================] - 333s 1s/step - loss: 0.1777 - accuracy: 0.9520 - val_loss: 0.0293 - val_accuracy: 0.99

In [89]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

9
Epoch 1/10
274/274 [==============================] - 362s 1s/step - loss: 0.3387 - accuracy: 0.9184 - val_loss: 0.0216 - val_accuracy: 0.9954
Epoch 2/10
274/274 [==============================] - 371s 1s/step - loss: 0.2200 - accuracy: 0.9436 - val_loss: 0.0442 - val_accuracy: 0.9911
Epoch 3/10
274/274 [==============================] - 373s 1s/step - loss: 0.1945 - accuracy: 0.9466 - val_loss: 0.0289 - val_accuracy: 0.9931
Epoch 4/10
274/274 [==============================] - 331s 1s/step - loss: 0.1646 - accuracy: 0.9553 - val_loss: 0.0161 - val_accuracy: 0.9959
Epoch 5/10
274/274 [==============================] - 326s 1s/step - loss: 0.1678 - accuracy: 0.9550 - val_loss: 0.0201 - val_accuracy: 0.9967
Epoch 6/10
274/274 [==============================] - 347s 1s/step - loss: 0.1622 - accuracy: 0.9574 - val_loss: 0.0185 - val_accuracy: 0.9972
Epoch 7/10
274/274 [==============================] - 363s 1s/step - loss: 0.1645 - accuracy: 0.9589 - val_loss: 0.0272 - val_accuracy: 0.99

In [90]:
print(n)#10
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

10
Epoch 1/10
301/301 [==============================] - 398s 1s/step - loss: 0.3121 - accuracy: 0.9230 - val_loss: 0.0611 - val_accuracy: 0.9918
Epoch 2/10
301/301 [==============================] - 397s 1s/step - loss: 0.2177 - accuracy: 0.9438 - val_loss: 0.0201 - val_accuracy: 0.9944
Epoch 3/10
301/301 [==============================] - 398s 1s/step - loss: 0.1973 - accuracy: 0.9507 - val_loss: 0.0347 - val_accuracy: 0.9944
Epoch 4/10
301/301 [==============================] - 389s 1s/step - loss: 0.1850 - accuracy: 0.9538 - val_loss: 0.0616 - val_accuracy: 0.9837
Epoch 5/10
301/301 [==============================] - 394s 1s/step - loss: 0.1627 - accuracy: 0.9562 - val_loss: 0.0201 - val_accuracy: 0.9962
Epoch 6/10
301/301 [==============================] - 404s 1s/step - loss: 0.1652 - accuracy: 0.9568 - val_loss: 0.0199 - val_accuracy: 0.9964
Epoch 7/10
301/301 [==============================] - 394s 1s/step - loss: 0.1433 - accuracy: 0.9629 - val_loss: 0.0826 - val_accuracy: 0.9

In [91]:
n=11
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

11
Epoch 1/10
329/329 [==============================] - 432s 1s/step - loss: 0.3175 - accuracy: 0.9293 - val_loss: 0.0157 - val_accuracy: 0.9954
Epoch 2/10
329/329 [==============================] - 424s 1s/step - loss: 0.1990 - accuracy: 0.9483 - val_loss: 0.0289 - val_accuracy: 0.9952
Epoch 3/10
329/329 [==============================] - 420s 1s/step - loss: 0.1647 - accuracy: 0.9550 - val_loss: 0.0228 - val_accuracy: 0.9939
Epoch 4/10
329/329 [==============================] - 422s 1s/step - loss: 0.1584 - accuracy: 0.9590 - val_loss: 0.0638 - val_accuracy: 0.9844
Epoch 5/10
329/329 [==============================] - 423s 1s/step - loss: 0.1548 - accuracy: 0.9588 - val_loss: 0.6519 - val_accuracy: 0.8831
Epoch 6/10
329/329 [==============================] - 422s 1s/step - loss: 0.1653 - accuracy: 0.9572 - val_loss: 0.0155 - val_accuracy: 0.9949
Epoch 7/10
329/329 [==============================] - 378s 1s/step - loss: 0.1371 - accuracy: 0.9658 - val_loss: 0.0915 - val_accuracy: 0.9

In [22]:
n=12
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

12
Epoch 1/10
356/356 [==============================] - 205s 426ms/step - loss: 0.2689 - accuracy: 0.9360 - val_loss: 0.0257 - val_accuracy: 0.9944
Epoch 2/10
356/356 [==============================] - 142s 400ms/step - loss: 0.1916 - accuracy: 0.9522 - val_loss: 0.0203 - val_accuracy: 0.9944
Epoch 3/10
356/356 [==============================] - 156s 437ms/step - loss: 0.1652 - accuracy: 0.9557 - val_loss: 0.0171 - val_accuracy: 0.9952
Epoch 4/10
356/356 [==============================] - 155s 436ms/step - loss: 0.1511 - accuracy: 0.9610 - val_loss: 0.0184 - val_accuracy: 0.9964
Epoch 5/10
356/356 [==============================] - 155s 434ms/step - loss: 0.1509 - accuracy: 0.9609 - val_loss: 0.0333 - val_accuracy: 0.9934
Epoch 6/10
356/356 [==============================] - 150s 421ms/step - loss: 0.1461 - accuracy: 0.9618 - val_loss: 0.0248 - val_accuracy: 0.9936
Epoch 7/10
356/356 [==============================] - 146s 411ms/step - loss: 0.1396 - accuracy: 0.9649 - val_loss: 0.026

In [23]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

13
Epoch 1/10
383/383 [==============================] - 173s 446ms/step - loss: 0.2462 - accuracy: 0.9411 - val_loss: 0.0204 - val_accuracy: 0.9957
Epoch 2/10
383/383 [==============================] - 179s 467ms/step - loss: 0.1795 - accuracy: 0.9544 - val_loss: 0.0171 - val_accuracy: 0.9949
Epoch 3/10
383/383 [==============================] - 176s 458ms/step - loss: 0.1697 - accuracy: 0.9560 - val_loss: 0.0224 - val_accuracy: 0.9959
Epoch 4/10
383/383 [==============================] - 175s 458ms/step - loss: 0.1419 - accuracy: 0.9637 - val_loss: 0.0210 - val_accuracy: 0.9944
Epoch 5/10
383/383 [==============================] - 176s 459ms/step - loss: 0.1428 - accuracy: 0.9637 - val_loss: 0.0178 - val_accuracy: 0.9941
Epoch 6/10
383/383 [==============================] - 177s 462ms/step - loss: 0.1436 - accuracy: 0.9630 - val_loss: 0.0403 - val_accuracy: 0.9923
Epoch 7/10
383/383 [==============================] - 176s 460ms/step - loss: 0.1384 - accuracy: 0.9670 - val_loss: 0.035

In [24]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

14
Epoch 1/10
411/411 [==============================] - 193s 462ms/step - loss: 0.2567 - accuracy: 0.9404 - val_loss: 0.0157 - val_accuracy: 0.9964
Epoch 2/10
411/411 [==============================] - 189s 459ms/step - loss: 0.1600 - accuracy: 0.9581 - val_loss: 0.0232 - val_accuracy: 0.9934
Epoch 3/10
411/411 [==============================] - 189s 459ms/step - loss: 0.1573 - accuracy: 0.9588 - val_loss: 0.0285 - val_accuracy: 0.9949
Epoch 4/10
411/411 [==============================] - 188s 458ms/step - loss: 0.1505 - accuracy: 0.9608 - val_loss: 0.0487 - val_accuracy: 0.9906
Epoch 5/10
411/411 [==============================] - 188s 457ms/step - loss: 0.1484 - accuracy: 0.9616 - val_loss: 0.0412 - val_accuracy: 0.9900
Epoch 6/10
411/411 [==============================] - 188s 458ms/step - loss: 0.1379 - accuracy: 0.9656 - val_loss: 0.0262 - val_accuracy: 0.9941
Epoch 7/10
411/411 [==============================] - 188s 458ms/step - loss: 0.1455 - accuracy: 0.9650 - val_loss: 0.016

In [25]:
print(n)#15
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

15
Epoch 1/10
438/438 [==============================] - 206s 463ms/step - loss: 0.2187 - accuracy: 0.9496 - val_loss: 0.0384 - val_accuracy: 0.9911
Epoch 2/10
438/438 [==============================] - 200s 457ms/step - loss: 0.1663 - accuracy: 0.9569 - val_loss: 0.0244 - val_accuracy: 0.9931
Epoch 3/10
438/438 [==============================] - 201s 458ms/step - loss: 0.1483 - accuracy: 0.9616 - val_loss: 0.0596 - val_accuracy: 0.9878
Epoch 4/10
438/438 [==============================] - 200s 456ms/step - loss: 0.1357 - accuracy: 0.9650 - val_loss: 0.0445 - val_accuracy: 0.9936
Epoch 5/10
438/438 [==============================] - 200s 458ms/step - loss: 0.1405 - accuracy: 0.9647 - val_loss: 0.0346 - val_accuracy: 0.9946
Epoch 6/10
438/438 [==============================] - 201s 459ms/step - loss: 0.1324 - accuracy: 0.9662 - val_loss: 0.0312 - val_accuracy: 0.9913
Epoch 7/10
438/438 [==============================] - 200s 456ms/step - loss: 0.1373 - accuracy: 0.9662 - val_loss: 0.020

In [26]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

16
Epoch 1/10
465/465 [==============================] - 211s 447ms/step - loss: 0.2137 - accuracy: 0.9502 - val_loss: 0.0420 - val_accuracy: 0.9908
Epoch 2/10
465/465 [==============================] - 196s 421ms/step - loss: 0.1519 - accuracy: 0.9599 - val_loss: 0.0242 - val_accuracy: 0.9952
Epoch 3/10
465/465 [==============================] - 196s 420ms/step - loss: 0.1410 - accuracy: 0.9647 - val_loss: 0.0219 - val_accuracy: 0.9952
Epoch 4/10
465/465 [==============================] - 196s 422ms/step - loss: 0.1412 - accuracy: 0.9650 - val_loss: 0.0246 - val_accuracy: 0.9969
Epoch 5/10
465/465 [==============================] - 196s 422ms/step - loss: 0.1307 - accuracy: 0.9676 - val_loss: 0.0248 - val_accuracy: 0.9952
Epoch 6/10
465/465 [==============================] - 196s 421ms/step - loss: 0.1417 - accuracy: 0.9662 - val_loss: 0.0375 - val_accuracy: 0.9923
Epoch 7/10
465/465 [==============================] - 195s 420ms/step - loss: 0.1390 - accuracy: 0.9662 - val_loss: 0.025

In [27]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

17
Epoch 1/10
493/493 [==============================] - 229s 458ms/step - loss: 0.2058 - accuracy: 0.9539 - val_loss: 0.0446 - val_accuracy: 0.9890
Epoch 2/10
493/493 [==============================] - 225s 456ms/step - loss: 0.1537 - accuracy: 0.9594 - val_loss: 0.0350 - val_accuracy: 0.9949
Epoch 3/10
493/493 [==============================] - 225s 457ms/step - loss: 0.1439 - accuracy: 0.9655 - val_loss: 0.0218 - val_accuracy: 0.9952
Epoch 4/10
493/493 [==============================] - 229s 464ms/step - loss: 0.1362 - accuracy: 0.9668 - val_loss: 0.0211 - val_accuracy: 0.9959
Epoch 5/10
493/493 [==============================] - 234s 474ms/step - loss: 0.1292 - accuracy: 0.9689 - val_loss: 0.0292 - val_accuracy: 0.9929
Epoch 6/10
493/493 [==============================] - 231s 468ms/step - loss: 0.1374 - accuracy: 0.9671 - val_loss: 0.0152 - val_accuracy: 0.9962
Epoch 7/10
493/493 [==============================] - 226s 458ms/step - loss: 0.1331 - accuracy: 0.9688 - val_loss: 0.020

In [28]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

18
Epoch 1/10
520/520 [==============================] - 241s 457ms/step - loss: 0.2112 - accuracy: 0.9531 - val_loss: 0.0213 - val_accuracy: 0.9946
Epoch 2/10
520/520 [==============================] - 238s 458ms/step - loss: 0.1463 - accuracy: 0.9632 - val_loss: 0.0223 - val_accuracy: 0.9944
Epoch 3/10
520/520 [==============================] - 237s 456ms/step - loss: 0.1370 - accuracy: 0.9658 - val_loss: 0.0220 - val_accuracy: 0.9954
Epoch 4/10
520/520 [==============================] - 237s 456ms/step - loss: 0.1453 - accuracy: 0.9654 - val_loss: 0.0241 - val_accuracy: 0.9941
Epoch 5/10
520/520 [==============================] - 237s 456ms/step - loss: 0.1344 - accuracy: 0.9679 - val_loss: 0.0200 - val_accuracy: 0.9952
Epoch 6/10
520/520 [==============================] - 239s 460ms/step - loss: 0.1358 - accuracy: 0.9674 - val_loss: 0.0206 - val_accuracy: 0.9949
Epoch 7/10
520/520 [==============================] - 238s 457ms/step - loss: 0.1288 - accuracy: 0.9703 - val_loss: 0.023

In [29]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

19
Epoch 1/10
547/547 [==============================] - 232s 420ms/step - loss: 0.1881 - accuracy: 0.9564 - val_loss: 0.0261 - val_accuracy: 0.9921
Epoch 2/10
547/547 [==============================] - 229s 418ms/step - loss: 0.1387 - accuracy: 0.9647 - val_loss: 0.0207 - val_accuracy: 0.9949
Epoch 3/10
547/547 [==============================] - 229s 419ms/step - loss: 0.1277 - accuracy: 0.9680 - val_loss: 0.0131 - val_accuracy: 0.9967
Epoch 4/10
547/547 [==============================] - 229s 419ms/step - loss: 0.1248 - accuracy: 0.9668 - val_loss: 0.0211 - val_accuracy: 0.9959
Epoch 5/10
547/547 [==============================] - 236s 431ms/step - loss: 0.1221 - accuracy: 0.9708 - val_loss: 0.0198 - val_accuracy: 0.9954
Epoch 6/10
547/547 [==============================] - 250s 456ms/step - loss: 0.1223 - accuracy: 0.9698 - val_loss: 0.0210 - val_accuracy: 0.9946
Epoch 7/10
547/547 [==============================] - 229s 418ms/step - loss: 0.1291 - accuracy: 0.9685 - val_loss: 0.030

In [30]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

20
Epoch 1/10
552/552 [==============================] - 236s 423ms/step - loss: 0.2042 - accuracy: 0.9535 - val_loss: 0.0344 - val_accuracy: 0.9918
Epoch 2/10
552/552 [==============================] - 232s 420ms/step - loss: 0.1411 - accuracy: 0.9644 - val_loss: 0.0363 - val_accuracy: 0.9931
Epoch 3/10
552/552 [==============================] - 243s 440ms/step - loss: 0.1320 - accuracy: 0.9666 - val_loss: 0.0342 - val_accuracy: 0.9944
Epoch 4/10
552/552 [==============================] - 251s 456ms/step - loss: 0.1226 - accuracy: 0.9692 - val_loss: 0.0222 - val_accuracy: 0.9959
Epoch 5/10
552/552 [==============================] - 243s 440ms/step - loss: 0.1301 - accuracy: 0.9683 - val_loss: 0.0334 - val_accuracy: 0.9936
Epoch 6/10
552/552 [==============================] - 232s 420ms/step - loss: 0.1334 - accuracy: 0.9683 - val_loss: 0.0338 - val_accuracy: 0.9941
Epoch 7/10
552/552 [==============================] - 231s 419ms/step - loss: 0.1329 - accuracy: 0.9689 - val_loss: 0.029

In [ ]:

new_model_dg_dir  = "D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_dg_e1"

i=12

for model in models_dg[12:]:
    model.save(new_model_dg_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_dg_e1_12\assets
Model has been saved


In [ ]:
loading = False

models_dg = []

if loading:
    for i in range(5):
        model_dg_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_gn_e1_"+str(i)
        print(model_dg_dir)
        model =utils.My_model('gtsrb',True,model_dg_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_dg.append(model)
       
    

In [ ]:
try:
    del deep_gini_values
    del top_images_by_dg
    del top_labels_by_dg
    del image_sets_dg
    del label_sets_dg
    del models_dg
except:
    print("Error")

In [ ]:
import gc
gc.collect()

### Softmax 

In [17]:
#softmax values
se_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/softmax_values.npy"
se_values = np.load(se_direction)


In [18]:
# Obtaining top n images by LSA values
top_images_by_se  = utils.get_x_of_indexes(list(np.flip(np.argsort(se_values))),x_train_and_adversary)
top_labels_by_se = utils.get_x_of_indexes(list(np.flip(np.argsort(se_values))),y_train_and_adversary)

In [19]:
m = n_data_points

n = 0
image_sets_se = []
label_sets_se = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_se)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_se)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_se)%m))
        top_images_by_se_n = np.array(top_images_by_se[:n+m+(len(top_images_by_se)%m)])
        top_labels_by_se_n = np.array(top_labels_by_se[:n+m+(len(top_images_by_se)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_se_n = np.array(top_images_by_se[:n+m])
        top_labels_by_se_n = np.array(top_labels_by_se[:n+m])
    image_sets_se.append(top_images_by_se_n)
    label_sets_se.append(top_labels_by_se_n)
    print(len(top_images_by_se_n))
    n += m



0 :
0  ->  1750
1750
1 :
0  ->  3500
3500
2 :
0  ->  5250
5250
3 :
0  ->  7000
7000
4 :
0  ->  8750
8750
5 :
0  ->  10500
10500
6 :
0  ->  12250
12250
7 :
0  ->  14000
14000
8 :
0  ->  15750
15750
9 :
0  ->  17500
17500
10 :
0  ->  19250
19250
11 :
0  ->  21000
21000
12 :
0  ->  22750
22750
13 :
0  ->  24500
24500
14 :
0  ->  26250
26250
15 :
0  ->  28000
28000
16 :
0  ->  29750
29750
17 :
0  ->  31500
31500
18 :
0  ->  33250
33250
19 :
0  ->  35000
35000
20 :
Last
0  ->  37037
35287


In [20]:
print(model_dir)
models_se = []
for i in range(len(label_sets_se)):
    print(i,":")
    model = utils.My_model(dataset,True, model_dir)
    model.compile_model()
    models_se.append(model)

C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06/
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled
20 :
Model loaded correctly
Model compiled


In [21]:
n=0
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

0
Epoch 1/10
28/28 [==============================] - 46s 565ms/step - loss: 2.8955 - accuracy: 0.3360 - val_loss: 0.0431 - val_accuracy: 0.9898
Epoch 2/10
28/28 [==============================] - 16s 560ms/step - loss: 1.6643 - accuracy: 0.5391 - val_loss: 0.0281 - val_accuracy: 0.9923
Epoch 3/10
28/28 [==============================] - 15s 558ms/step - loss: 1.2584 - accuracy: 0.6345 - val_loss: 0.0542 - val_accuracy: 0.9862
Epoch 4/10
28/28 [==============================] - 15s 533ms/step - loss: 1.0958 - accuracy: 0.6793 - val_loss: 0.0390 - val_accuracy: 0.9906
Epoch 5/10
28/28 [==============================] - 14s 520ms/step - loss: 0.8742 - accuracy: 0.7473 - val_loss: 0.0421 - val_accuracy: 0.9890
Epoch 6/10
28/28 [==============================] - 14s 519ms/step - loss: 0.8162 - accuracy: 0.7591 - val_loss: 0.0662 - val_accuracy: 0.9832
Epoch 7/10
28/28 [==============================] - 14s 513ms/step - loss: 0.7226 - accuracy: 0.7935 - val_loss: 0.1209 - val_accuracy: 0.97

In [22]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

1
Epoch 1/10
55/55 [==============================] - 26s 444ms/step - loss: 1.5617 - accuracy: 0.6303 - val_loss: 0.0266 - val_accuracy: 0.9946
Epoch 2/10
55/55 [==============================] - 24s 446ms/step - loss: 0.9600 - accuracy: 0.7546 - val_loss: 0.0154 - val_accuracy: 0.9954
Epoch 3/10
55/55 [==============================] - 24s 438ms/step - loss: 0.7111 - accuracy: 0.8017 - val_loss: 0.0390 - val_accuracy: 0.9936
Epoch 4/10
55/55 [==============================] - 24s 443ms/step - loss: 0.6581 - accuracy: 0.8186 - val_loss: 0.0209 - val_accuracy: 0.9954
Epoch 5/10
55/55 [==============================] - 25s 464ms/step - loss: 0.5885 - accuracy: 0.8372 - val_loss: 0.0166 - val_accuracy: 0.9969
Epoch 6/10
55/55 [==============================] - 25s 452ms/step - loss: 0.5251 - accuracy: 0.8543 - val_loss: 0.0128 - val_accuracy: 0.9957
Epoch 7/10
55/55 [==============================] - 24s 437ms/step - loss: 0.4655 - accuracy: 0.8700 - val_loss: 0.0144 - val_accuracy: 0.99

In [23]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

2
Epoch 1/10
83/83 [==============================] - 37s 420ms/step - loss: 1.1055 - accuracy: 0.7487 - val_loss: 0.0281 - val_accuracy: 0.9946
Epoch 2/10
83/83 [==============================] - 35s 420ms/step - loss: 0.6118 - accuracy: 0.8321 - val_loss: 0.0198 - val_accuracy: 0.9959
Epoch 3/10
83/83 [==============================] - 35s 426ms/step - loss: 0.4941 - accuracy: 0.8559 - val_loss: 0.0266 - val_accuracy: 0.9941
Epoch 4/10
83/83 [==============================] - 36s 431ms/step - loss: 0.4236 - accuracy: 0.8842 - val_loss: 0.0210 - val_accuracy: 0.9949
Epoch 5/10
83/83 [==============================] - 36s 436ms/step - loss: 0.3955 - accuracy: 0.8852 - val_loss: 0.0181 - val_accuracy: 0.9964
Epoch 6/10
83/83 [==============================] - 37s 442ms/step - loss: 0.3228 - accuracy: 0.9154 - val_loss: 0.1208 - val_accuracy: 0.9806
Epoch 7/10
83/83 [==============================] - 37s 447ms/step - loss: 0.3526 - accuracy: 0.8950 - val_loss: 0.0146 - val_accuracy: 0.99

In [24]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

3
Epoch 1/10
110/110 [==============================] - 52s 458ms/step - loss: 0.8791 - accuracy: 0.8024 - val_loss: 0.0205 - val_accuracy: 0.9957
Epoch 2/10
110/110 [==============================] - 50s 452ms/step - loss: 0.4640 - accuracy: 0.8743 - val_loss: 0.0221 - val_accuracy: 0.9946
Epoch 3/10
110/110 [==============================] - 50s 455ms/step - loss: 0.3873 - accuracy: 0.8926 - val_loss: 0.0376 - val_accuracy: 0.9895
Epoch 4/10
110/110 [==============================] - 50s 454ms/step - loss: 0.3342 - accuracy: 0.9102 - val_loss: 0.0222 - val_accuracy: 0.9949
Epoch 5/10
110/110 [==============================] - 50s 455ms/step - loss: 0.3169 - accuracy: 0.9142 - val_loss: 0.0137 - val_accuracy: 0.9959
Epoch 6/10
110/110 [==============================] - 50s 453ms/step - loss: 0.2861 - accuracy: 0.9169 - val_loss: 0.0235 - val_accuracy: 0.9946
Epoch 7/10
110/110 [==============================] - 50s 456ms/step - loss: 0.2698 - accuracy: 0.9202 - val_loss: 0.0174 - val_

In [25]:
print(n)#4
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

4
Epoch 1/10
137/137 [==============================] - 63s 449ms/step - loss: 0.6958 - accuracy: 0.8390 - val_loss: 0.0243 - val_accuracy: 0.9939
Epoch 2/10
137/137 [==============================] - 61s 446ms/step - loss: 0.3987 - accuracy: 0.8857 - val_loss: 0.0163 - val_accuracy: 0.9959
Epoch 3/10
137/137 [==============================] - 61s 446ms/step - loss: 0.3507 - accuracy: 0.9066 - val_loss: 0.0070 - val_accuracy: 0.9980
Epoch 4/10
137/137 [==============================] - 62s 454ms/step - loss: 0.3018 - accuracy: 0.9206 - val_loss: 0.0120 - val_accuracy: 0.9972
Epoch 5/10
137/137 [==============================] - 67s 487ms/step - loss: 0.2497 - accuracy: 0.9267 - val_loss: 0.0197 - val_accuracy: 0.9967
Epoch 6/10
137/137 [==============================] - 67s 489ms/step - loss: 0.2225 - accuracy: 0.9322 - val_loss: 0.0167 - val_accuracy: 0.9964
Epoch 7/10
137/137 [==============================] - 67s 487ms/step - loss: 0.2403 - accuracy: 0.9286 - val_loss: 0.0126 - val_

In [26]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

5
Epoch 1/10
165/165 [==============================] - 82s 484ms/step - loss: 0.5563 - accuracy: 0.8682 - val_loss: 0.0240 - val_accuracy: 0.9936
Epoch 2/10
165/165 [==============================] - 79s 480ms/step - loss: 0.3613 - accuracy: 0.9027 - val_loss: 0.0375 - val_accuracy: 0.9934
Epoch 3/10
165/165 [==============================] - 82s 495ms/step - loss: 0.3170 - accuracy: 0.9119 - val_loss: 0.0166 - val_accuracy: 0.9962
Epoch 4/10
165/165 [==============================] - 84s 507ms/step - loss: 0.2686 - accuracy: 0.9271 - val_loss: 0.0121 - val_accuracy: 0.9980
Epoch 5/10
165/165 [==============================] - 81s 488ms/step - loss: 0.2281 - accuracy: 0.9362 - val_loss: 0.0253 - val_accuracy: 0.9959
Epoch 6/10
165/165 [==============================] - 79s 481ms/step - loss: 0.2102 - accuracy: 0.9453 - val_loss: 0.0395 - val_accuracy: 0.9941
Epoch 7/10
165/165 [==============================] - 79s 479ms/step - loss: 0.2109 - accuracy: 0.9407 - val_loss: 0.0272 - val_

In [27]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

6
Epoch 1/10
192/192 [==============================] - 95s 482ms/step - loss: 0.4792 - accuracy: 0.8927 - val_loss: 0.1509 - val_accuracy: 0.9793
Epoch 2/10
192/192 [==============================] - 92s 478ms/step - loss: 0.2994 - accuracy: 0.9227 - val_loss: 0.0229 - val_accuracy: 0.9941
Epoch 3/10
192/192 [==============================] - 92s 482ms/step - loss: 0.2743 - accuracy: 0.9281 - val_loss: 0.0215 - val_accuracy: 0.9954
Epoch 4/10
192/192 [==============================] - 95s 494ms/step - loss: 0.2391 - accuracy: 0.9357 - val_loss: 0.0237 - val_accuracy: 0.9957
Epoch 5/10
192/192 [==============================] - 92s 477ms/step - loss: 0.2265 - accuracy: 0.9368 - val_loss: 0.0190 - val_accuracy: 0.9946
Epoch 6/10
192/192 [==============================] - 92s 480ms/step - loss: 0.1942 - accuracy: 0.9460 - val_loss: 0.0267 - val_accuracy: 0.9944
Epoch 7/10
192/192 [==============================] - 92s 480ms/step - loss: 0.1908 - accuracy: 0.9475 - val_loss: 0.0177 - val_

In [28]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

7
Epoch 1/10
219/219 [==============================] - 106s 476ms/step - loss: 0.4054 - accuracy: 0.9010 - val_loss: 0.0246 - val_accuracy: 0.9941
Epoch 2/10
219/219 [==============================] - 104s 475ms/step - loss: 0.2713 - accuracy: 0.9284 - val_loss: 0.0258 - val_accuracy: 0.9949
Epoch 3/10
219/219 [==============================] - 104s 476ms/step - loss: 0.2401 - accuracy: 0.9365 - val_loss: 0.0152 - val_accuracy: 0.9964
Epoch 4/10
219/219 [==============================] - 104s 475ms/step - loss: 0.2215 - accuracy: 0.9399 - val_loss: 0.0226 - val_accuracy: 0.9957
Epoch 5/10
219/219 [==============================] - 104s 475ms/step - loss: 0.1922 - accuracy: 0.9484 - val_loss: 0.0216 - val_accuracy: 0.9946
Epoch 6/10
219/219 [==============================] - 104s 474ms/step - loss: 0.1945 - accuracy: 0.9458 - val_loss: 0.0180 - val_accuracy: 0.9967
Epoch 7/10
219/219 [==============================] - 104s 474ms/step - loss: 0.1730 - accuracy: 0.9523 - val_loss: 0.0234

In [29]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

8
Epoch 1/10
247/247 [==============================] - 119s 472ms/step - loss: 0.3865 - accuracy: 0.9102 - val_loss: 0.0161 - val_accuracy: 0.9952
Epoch 2/10
247/247 [==============================] - 117s 473ms/step - loss: 0.2459 - accuracy: 0.9358 - val_loss: 0.0114 - val_accuracy: 0.9969
Epoch 3/10
247/247 [==============================] - 117s 473ms/step - loss: 0.2147 - accuracy: 0.9422 - val_loss: 0.0219 - val_accuracy: 0.9969
Epoch 4/10
247/247 [==============================] - 117s 473ms/step - loss: 0.1954 - accuracy: 0.9501 - val_loss: 0.0409 - val_accuracy: 0.9911
Epoch 5/10
247/247 [==============================] - 116s 471ms/step - loss: 0.1895 - accuracy: 0.9510 - val_loss: 0.0243 - val_accuracy: 0.9962
Epoch 6/10
247/247 [==============================] - 117s 473ms/step - loss: 0.1822 - accuracy: 0.9527 - val_loss: 0.0530 - val_accuracy: 0.9842
Epoch 7/10
247/247 [==============================] - 116s 472ms/step - loss: 0.1791 - accuracy: 0.9523 - val_loss: 0.0494

In [30]:
print(n)#9
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

9
Epoch 1/10
274/274 [==============================] - 133s 479ms/step - loss: 0.3365 - accuracy: 0.9191 - val_loss: 0.0452 - val_accuracy: 0.9895
Epoch 2/10
274/274 [==============================] - 128s 466ms/step - loss: 0.2301 - accuracy: 0.9389 - val_loss: 0.0471 - val_accuracy: 0.9931
Epoch 3/10
274/274 [==============================] - 128s 468ms/step - loss: 0.2079 - accuracy: 0.9426 - val_loss: 0.0284 - val_accuracy: 0.9934
Epoch 4/10
274/274 [==============================] - 129s 471ms/step - loss: 0.1773 - accuracy: 0.9521 - val_loss: 0.0174 - val_accuracy: 0.9957
Epoch 5/10
274/274 [==============================] - 128s 467ms/step - loss: 0.1773 - accuracy: 0.9546 - val_loss: 0.0247 - val_accuracy: 0.9949
Epoch 6/10
274/274 [==============================] - 128s 467ms/step - loss: 0.1769 - accuracy: 0.9532 - val_loss: 0.0213 - val_accuracy: 0.9964
Epoch 7/10
274/274 [==============================] - 128s 469ms/step - loss: 0.1725 - accuracy: 0.9560 - val_loss: 0.0216

In [31]:
n=10
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

10
Epoch 1/10
301/301 [==============================] - 132s 432ms/step - loss: 0.3091 - accuracy: 0.9218 - val_loss: 0.0350 - val_accuracy: 0.9903
Epoch 2/10
301/301 [==============================] - 129s 427ms/step - loss: 0.2266 - accuracy: 0.9391 - val_loss: 0.0241 - val_accuracy: 0.9959
Epoch 3/10
301/301 [==============================] - 128s 426ms/step - loss: 0.1929 - accuracy: 0.9505 - val_loss: 0.0685 - val_accuracy: 0.9865
Epoch 4/10
301/301 [==============================] - 129s 428ms/step - loss: 0.1729 - accuracy: 0.9533 - val_loss: 0.0229 - val_accuracy: 0.9957
Epoch 5/10
301/301 [==============================] - 128s 427ms/step - loss: 0.1554 - accuracy: 0.9588 - val_loss: 0.0164 - val_accuracy: 0.9969
Epoch 6/10
301/301 [==============================] - 128s 426ms/step - loss: 0.1690 - accuracy: 0.9597 - val_loss: 0.0259 - val_accuracy: 0.9962
Epoch 7/10
301/301 [==============================] - 128s 426ms/step - loss: 0.1524 - accuracy: 0.9597 - val_loss: 0.021

In [32]:
n=11
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

11
Epoch 1/10
329/329 [==============================] - 143s 429ms/step - loss: 0.2836 - accuracy: 0.9351 - val_loss: 0.0303 - val_accuracy: 0.9936
Epoch 2/10
329/329 [==============================] - 152s 462ms/step - loss: 0.2014 - accuracy: 0.9482 - val_loss: 0.0386 - val_accuracy: 0.9931
Epoch 3/10
329/329 [==============================] - 152s 462ms/step - loss: 0.1792 - accuracy: 0.9514 - val_loss: 0.0182 - val_accuracy: 0.9962
Epoch 4/10
329/329 [==============================] - 152s 463ms/step - loss: 0.1709 - accuracy: 0.9564 - val_loss: 0.0143 - val_accuracy: 0.9967
Epoch 5/10
329/329 [==============================] - 152s 462ms/step - loss: 0.1528 - accuracy: 0.9596 - val_loss: 0.0188 - val_accuracy: 0.9952
Epoch 6/10
329/329 [==============================] - 152s 462ms/step - loss: 0.1461 - accuracy: 0.9639 - val_loss: 0.0359 - val_accuracy: 0.9957
Epoch 7/10
329/329 [==============================] - 155s 471ms/step - loss: 0.1499 - accuracy: 0.9616 - val_loss: 0.011

In [33]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

12
Epoch 1/10
356/356 [==============================] - 167s 462ms/step - loss: 0.2752 - accuracy: 0.9373 - val_loss: 0.0192 - val_accuracy: 0.9954
Epoch 2/10
356/356 [==============================] - 165s 462ms/step - loss: 0.1804 - accuracy: 0.9529 - val_loss: 0.0206 - val_accuracy: 0.9949
Epoch 3/10
356/356 [==============================] - 165s 464ms/step - loss: 0.1777 - accuracy: 0.9551 - val_loss: 0.0211 - val_accuracy: 0.9957
Epoch 4/10
356/356 [==============================] - 164s 462ms/step - loss: 0.1601 - accuracy: 0.9581 - val_loss: 0.0237 - val_accuracy: 0.9959
Epoch 5/10
356/356 [==============================] - 164s 461ms/step - loss: 0.1437 - accuracy: 0.9629 - val_loss: 0.0502 - val_accuracy: 0.9913
Epoch 6/10
356/356 [==============================] - 165s 464ms/step - loss: 0.1428 - accuracy: 0.9638 - val_loss: 0.0304 - val_accuracy: 0.9934
Epoch 7/10
356/356 [==============================] - 164s 461ms/step - loss: 0.1391 - accuracy: 0.9663 - val_loss: 0.025

In [34]:
n=13
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

13
Epoch 1/10
383/383 [==============================] - 179s 461ms/step - loss: 0.2562 - accuracy: 0.9383 - val_loss: 0.0212 - val_accuracy: 0.9959
Epoch 2/10
383/383 [==============================] - 176s 461ms/step - loss: 0.1693 - accuracy: 0.9562 - val_loss: 0.0206 - val_accuracy: 0.9954
Epoch 3/10
383/383 [==============================] - 176s 460ms/step - loss: 0.1674 - accuracy: 0.9559 - val_loss: 0.0269 - val_accuracy: 0.9946
Epoch 4/10
383/383 [==============================] - 177s 462ms/step - loss: 0.1472 - accuracy: 0.9622 - val_loss: 0.0227 - val_accuracy: 0.9939
Epoch 5/10
383/383 [==============================] - 176s 460ms/step - loss: 0.1399 - accuracy: 0.9619 - val_loss: 0.0171 - val_accuracy: 0.9972
Epoch 6/10
383/383 [==============================] - 176s 460ms/step - loss: 0.1537 - accuracy: 0.9622 - val_loss: 0.0302 - val_accuracy: 0.9931
Epoch 7/10
383/383 [==============================] - 176s 461ms/step - loss: 0.1433 - accuracy: 0.9647 - val_loss: 0.028

In [35]:
print(n)#14
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

14
Epoch 1/10
411/411 [==============================] - 191s 460ms/step - loss: 0.2452 - accuracy: 0.9438 - val_loss: 0.0318 - val_accuracy: 0.9926
Epoch 2/10
411/411 [==============================] - 188s 458ms/step - loss: 0.1724 - accuracy: 0.9581 - val_loss: 0.0263 - val_accuracy: 0.9946
Epoch 3/10
411/411 [==============================] - 190s 462ms/step - loss: 0.1623 - accuracy: 0.9591 - val_loss: 0.0222 - val_accuracy: 0.9957
Epoch 4/10
411/411 [==============================] - 191s 465ms/step - loss: 0.1445 - accuracy: 0.9621 - val_loss: 0.0413 - val_accuracy: 0.9929
Epoch 5/10
411/411 [==============================] - 173s 420ms/step - loss: 0.1499 - accuracy: 0.9612 - val_loss: 0.0230 - val_accuracy: 0.9962
Epoch 6/10
411/411 [==============================] - 173s 422ms/step - loss: 0.1530 - accuracy: 0.9649 - val_loss: 0.0313 - val_accuracy: 0.9954
Epoch 7/10
411/411 [==============================] - 173s 422ms/step - loss: 0.1504 - accuracy: 0.9648 - val_loss: 0.011

In [36]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

15
Epoch 1/10
438/438 [==============================] - 186s 420ms/step - loss: 0.2144 - accuracy: 0.9489 - val_loss: 0.0229 - val_accuracy: 0.9954
Epoch 2/10
438/438 [==============================] - 184s 419ms/step - loss: 0.1559 - accuracy: 0.9596 - val_loss: 0.0259 - val_accuracy: 0.9954
Epoch 3/10
438/438 [==============================] - 194s 444ms/step - loss: 0.1444 - accuracy: 0.9627 - val_loss: 0.1109 - val_accuracy: 0.9890
Epoch 4/10
438/438 [==============================] - 201s 460ms/step - loss: 0.1546 - accuracy: 0.9615 - val_loss: 0.0303 - val_accuracy: 0.9949
Epoch 5/10
438/438 [==============================] - 201s 459ms/step - loss: 0.1343 - accuracy: 0.9662 - val_loss: 0.0338 - val_accuracy: 0.9934
Epoch 6/10
438/438 [==============================] - 200s 458ms/step - loss: 0.1552 - accuracy: 0.9630 - val_loss: 0.0276 - val_accuracy: 0.9959
Epoch 7/10
438/438 [==============================] - 203s 463ms/step - loss: 0.1530 - accuracy: 0.9633 - val_loss: 0.022

In [37]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

16
Epoch 1/10
465/465 [==============================] - 216s 460ms/step - loss: 0.2127 - accuracy: 0.9508 - val_loss: 0.0226 - val_accuracy: 0.9957
Epoch 2/10
465/465 [==============================] - 214s 459ms/step - loss: 0.1586 - accuracy: 0.9598 - val_loss: 0.0349 - val_accuracy: 0.9931
Epoch 3/10
465/465 [==============================] - 214s 460ms/step - loss: 0.1375 - accuracy: 0.9649 - val_loss: 0.0414 - val_accuracy: 0.9895
Epoch 4/10
465/465 [==============================] - 214s 460ms/step - loss: 0.1412 - accuracy: 0.9676 - val_loss: 0.0275 - val_accuracy: 0.9934
Epoch 5/10
465/465 [==============================] - 214s 460ms/step - loss: 0.1357 - accuracy: 0.9651 - val_loss: 0.0276 - val_accuracy: 0.9949
Epoch 6/10
465/465 [==============================] - 213s 458ms/step - loss: 0.1265 - accuracy: 0.9716 - val_loss: 0.0408 - val_accuracy: 0.9911
Epoch 7/10
465/465 [==============================] - 214s 461ms/step - loss: 0.1251 - accuracy: 0.9694 - val_loss: 0.015

In [38]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

17
Epoch 1/10
493/493 [==============================] - 228s 458ms/step - loss: 0.1928 - accuracy: 0.9544 - val_loss: 0.0258 - val_accuracy: 0.9929
Epoch 2/10
493/493 [==============================] - 225s 456ms/step - loss: 0.1536 - accuracy: 0.9608 - val_loss: 0.0169 - val_accuracy: 0.9967
Epoch 3/10
493/493 [==============================] - 226s 458ms/step - loss: 0.1377 - accuracy: 0.9654 - val_loss: 0.0298 - val_accuracy: 0.9952
Epoch 4/10
493/493 [==============================] - 218s 441ms/step - loss: 0.1410 - accuracy: 0.9653 - val_loss: 0.0245 - val_accuracy: 0.9934
Epoch 5/10
493/493 [==============================] - 208s 422ms/step - loss: 0.1315 - accuracy: 0.9665 - val_loss: 0.0776 - val_accuracy: 0.9811
Epoch 6/10
493/493 [==============================] - 207s 420ms/step - loss: 0.1424 - accuracy: 0.9660 - val_loss: 0.0581 - val_accuracy: 0.9934
Epoch 7/10
493/493 [==============================] - 221s 449ms/step - loss: 0.1500 - accuracy: 0.9666 - val_loss: 0.039

In [39]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

18
Epoch 1/10
520/520 [==============================] - 230s 440ms/step - loss: 0.1988 - accuracy: 0.9541 - val_loss: 0.0388 - val_accuracy: 0.9911
Epoch 2/10
520/520 [==============================] - 237s 456ms/step - loss: 0.1435 - accuracy: 0.9632 - val_loss: 0.0182 - val_accuracy: 0.9962
Epoch 3/10
520/520 [==============================] - 238s 458ms/step - loss: 0.1373 - accuracy: 0.9664 - val_loss: 0.0241 - val_accuracy: 0.9936
Epoch 4/10
520/520 [==============================] - 238s 457ms/step - loss: 0.1264 - accuracy: 0.9698 - val_loss: 0.0264 - val_accuracy: 0.9967
Epoch 5/10
520/520 [==============================] - 237s 455ms/step - loss: 0.1314 - accuracy: 0.9675 - val_loss: 0.0225 - val_accuracy: 0.9949
Epoch 6/10
520/520 [==============================] - 236s 454ms/step - loss: 0.1268 - accuracy: 0.9694 - val_loss: 0.0266 - val_accuracy: 0.9949
Epoch 7/10
520/520 [==============================] - 236s 455ms/step - loss: 0.1365 - accuracy: 0.9681 - val_loss: 0.018

In [40]:
print(n)#19
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

19
Epoch 1/10
547/547 [==============================] - 253s 458ms/step - loss: 0.1817 - accuracy: 0.9562 - val_loss: 0.0578 - val_accuracy: 0.9890
Epoch 2/10
547/547 [==============================] - 241s 441ms/step - loss: 0.1350 - accuracy: 0.9663 - val_loss: 0.0326 - val_accuracy: 0.9926
Epoch 3/10
547/547 [==============================] - 229s 419ms/step - loss: 0.1349 - accuracy: 0.9668 - val_loss: 0.0374 - val_accuracy: 0.9934
Epoch 4/10
547/547 [==============================] - 229s 419ms/step - loss: 0.1288 - accuracy: 0.9686 - val_loss: 0.0294 - val_accuracy: 0.9934
Epoch 5/10
547/547 [==============================] - 228s 417ms/step - loss: 0.1296 - accuracy: 0.9692 - val_loss: 0.0385 - val_accuracy: 0.9941
Epoch 6/10
547/547 [==============================] - 228s 417ms/step - loss: 0.1354 - accuracy: 0.9685 - val_loss: 0.0727 - val_accuracy: 0.9895
Epoch 7/10
547/547 [==============================] - 229s 418ms/step - loss: 0.1262 - accuracy: 0.9698 - val_loss: 0.025

In [41]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

20
Epoch 1/10
552/552 [==============================] - 254s 456ms/step - loss: 0.1741 - accuracy: 0.9598 - val_loss: 0.0273 - val_accuracy: 0.9954
Epoch 2/10
552/552 [==============================] - 252s 456ms/step - loss: 0.1357 - accuracy: 0.9659 - val_loss: 0.0221 - val_accuracy: 0.9959
Epoch 3/10
552/552 [==============================] - 251s 455ms/step - loss: 0.1305 - accuracy: 0.9655 - val_loss: 0.0332 - val_accuracy: 0.9952
Epoch 4/10
552/552 [==============================] - 251s 454ms/step - loss: 0.1366 - accuracy: 0.9683 - val_loss: 0.0532 - val_accuracy: 0.9900
Epoch 5/10
552/552 [==============================] - 253s 458ms/step - loss: 0.1343 - accuracy: 0.9696 - val_loss: 0.0361 - val_accuracy: 0.9903
Epoch 6/10
552/552 [==============================] - 251s 454ms/step - loss: 0.1340 - accuracy: 0.9699 - val_loss: 0.0404 - val_accuracy: 0.9929
Epoch 7/10
552/552 [==============================] - 251s 454ms/step - loss: 0.1315 - accuracy: 0.9702 - val_loss: 0.057

In [ ]:
len(models_se)

In [42]:
new_model_se_dir  = "D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_se_e1"

i=0

for model in models_se:
    model.save(new_model_se_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_se_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_se_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_se_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_se_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_se_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_se_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_se_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_se_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model

In [43]:
try:
    del se_values
    del top_images_by_se
    del top_labels_by_se
    del image_sets_se
    del label_sets_se
    del models_se
except:
    print("Error")

In [44]:
gc.collect()

632102

## Training guided by Random values

In [ ]:
import random
random_indexes =list(range(len(x_train_and_adversary)))
random.shuffle(random_indexes)
print(random_indexes[:10])
print(len(random_indexes))

In [45]:
save_dir = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/random_values.npy"


random_indexes = np.load(save_dir)

In [46]:
len(random_indexes)

35287

In [47]:
# Obtaining top n images by random values
top_images_by_random = utils.get_x_of_indexes(list(np.flip(np.argsort(random_indexes))),x_train_and_adversary)
top_labels_by_random = utils.get_x_of_indexes(list(np.flip(np.argsort(random_indexes))),y_train_and_adversary)

In [48]:
m = n_data_points
n = 0
image_sets_random = []
label_sets_random = []



for i in range((len(top_images_by_random)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_random)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_random)%m))
        top_images_by_random_n = np.array(top_images_by_random[:n+m+(len(top_images_by_random)%m)])
        top_labels_by_random_n = np.array(top_labels_by_random[:n+m+(len(top_images_by_random)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_random_n = np.array(top_images_by_random[:n+m])
        top_labels_by_random_n = np.array(top_labels_by_random[:n+m])
    image_sets_random.append(top_images_by_random_n)
    label_sets_random.append(top_labels_by_random_n)
    print(len(top_images_by_random_n))
    n += m




0 :
0  ->  1750
1750
1 :
0  ->  3500
3500
2 :
0  ->  5250
5250
3 :
0  ->  7000
7000
4 :
0  ->  8750
8750
5 :
0  ->  10500
10500
6 :
0  ->  12250
12250
7 :
0  ->  14000
14000
8 :
0  ->  15750
15750
9 :
0  ->  17500
17500
10 :
0  ->  19250
19250
11 :
0  ->  21000
21000
12 :
0  ->  22750
22750
13 :
0  ->  24500
24500
14 :
0  ->  26250
26250
15 :
0  ->  28000
28000
16 :
0  ->  29750
29750
17 :
0  ->  31500
31500
18 :
0  ->  33250
33250
19 :
0  ->  35000
35000
20 :
Last
0  ->  37037
35287


In [49]:
len(image_sets_random)

21

In [50]:
print(model_dir)

models_random = []
for i in range(len(label_sets_random)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_random.append(model)


C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06/
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled
20 :
Model loaded correctly
Model compiled


In [51]:
n=0
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

0
Epoch 1/10
28/28 [==============================] - 23s 765ms/step - loss: 0.2543 - accuracy: 0.9421 - val_loss: 0.0435 - val_accuracy: 0.9913
Epoch 2/10
28/28 [==============================] - 21s 742ms/step - loss: 0.1670 - accuracy: 0.9593 - val_loss: 0.0259 - val_accuracy: 0.9939
Epoch 3/10
28/28 [==============================] - 21s 750ms/step - loss: 0.1570 - accuracy: 0.9586 - val_loss: 0.0245 - val_accuracy: 0.9941
Epoch 4/10
28/28 [==============================] - 21s 745ms/step - loss: 0.1025 - accuracy: 0.9680 - val_loss: 0.0321 - val_accuracy: 0.9923
Epoch 5/10
28/28 [==============================] - 21s 748ms/step - loss: 0.1045 - accuracy: 0.9741 - val_loss: 0.0312 - val_accuracy: 0.9923
Epoch 6/10
28/28 [==============================] - 21s 751ms/step - loss: 0.1192 - accuracy: 0.9690 - val_loss: 0.0326 - val_accuracy: 0.9929
Epoch 7/10
28/28 [==============================] - 21s 746ms/step - loss: 0.0906 - accuracy: 0.9791 - val_loss: 0.0399 - val_accuracy: 0.99

In [52]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

1
Epoch 1/10
55/55 [==============================] - 39s 663ms/step - loss: 0.2814 - accuracy: 0.9485 - val_loss: 0.0447 - val_accuracy: 0.9931
Epoch 2/10
55/55 [==============================] - 36s 657ms/step - loss: 0.1250 - accuracy: 0.9688 - val_loss: 0.0295 - val_accuracy: 0.9931
Epoch 3/10
55/55 [==============================] - 36s 649ms/step - loss: 0.1301 - accuracy: 0.9674 - val_loss: 0.0304 - val_accuracy: 0.9944
Epoch 4/10
55/55 [==============================] - 36s 652ms/step - loss: 0.1490 - accuracy: 0.9618 - val_loss: 0.0377 - val_accuracy: 0.9890
Epoch 5/10
55/55 [==============================] - 36s 649ms/step - loss: 0.1064 - accuracy: 0.9707 - val_loss: 0.0535 - val_accuracy: 0.9860
Epoch 6/10
55/55 [==============================] - 36s 650ms/step - loss: 0.1068 - accuracy: 0.9727 - val_loss: 0.0264 - val_accuracy: 0.9949
Epoch 7/10
55/55 [==============================] - 36s 649ms/step - loss: 0.0790 - accuracy: 0.9801 - val_loss: 0.0309 - val_accuracy: 0.99

In [53]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

2
Epoch 1/10
83/83 [==============================] - 57s 660ms/step - loss: 0.2090 - accuracy: 0.9578 - val_loss: 0.0492 - val_accuracy: 0.9906
Epoch 2/10
83/83 [==============================] - 54s 653ms/step - loss: 0.1572 - accuracy: 0.9579 - val_loss: 0.0300 - val_accuracy: 0.9936
Epoch 3/10
83/83 [==============================] - 54s 650ms/step - loss: 0.1300 - accuracy: 0.9663 - val_loss: 0.0245 - val_accuracy: 0.9939
Epoch 4/10
83/83 [==============================] - 54s 651ms/step - loss: 0.1234 - accuracy: 0.9670 - val_loss: 0.0226 - val_accuracy: 0.9954
Epoch 5/10
83/83 [==============================] - 56s 676ms/step - loss: 0.1320 - accuracy: 0.9667 - val_loss: 0.0352 - val_accuracy: 0.9957
Epoch 6/10
83/83 [==============================] - 54s 646ms/step - loss: 0.1295 - accuracy: 0.9704 - val_loss: 0.0351 - val_accuracy: 0.9906
Epoch 7/10
83/83 [==============================] - 57s 682ms/step - loss: 0.1155 - accuracy: 0.9685 - val_loss: 0.0261 - val_accuracy: 0.99

In [54]:
n=3
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

3
Epoch 1/10
110/110 [==============================] - 73s 647ms/step - loss: 0.2509 - accuracy: 0.9525 - val_loss: 0.0265 - val_accuracy: 0.9936
Epoch 2/10
110/110 [==============================] - 73s 661ms/step - loss: 0.1484 - accuracy: 0.9590 - val_loss: 0.0317 - val_accuracy: 0.9934
Epoch 3/10
110/110 [==============================] - 73s 660ms/step - loss: 0.1391 - accuracy: 0.9653 - val_loss: 0.0412 - val_accuracy: 0.9906
Epoch 4/10
110/110 [==============================] - 73s 660ms/step - loss: 0.1338 - accuracy: 0.9635 - val_loss: 0.0387 - val_accuracy: 0.9929
Epoch 5/10
110/110 [==============================] - 73s 665ms/step - loss: 0.1213 - accuracy: 0.9676 - val_loss: 0.0244 - val_accuracy: 0.9954
Epoch 6/10
110/110 [==============================] - 84s 765ms/step - loss: 0.1162 - accuracy: 0.9699 - val_loss: 0.0283 - val_accuracy: 0.9952
Epoch 7/10
110/110 [==============================] - 107s 970ms/step - loss: 0.0816 - accuracy: 0.9779 - val_loss: 0.0340 - val

In [55]:
print(n)#4

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

4
Epoch 1/10
137/137 [==============================] - 136s 976ms/step - loss: 0.2372 - accuracy: 0.9453 - val_loss: 0.0253 - val_accuracy: 0.9939
Epoch 2/10
137/137 [==============================] - 128s 932ms/step - loss: 0.1625 - accuracy: 0.9592 - val_loss: 0.0256 - val_accuracy: 0.9941
Epoch 3/10
137/137 [==============================] - 128s 934ms/step - loss: 0.1500 - accuracy: 0.9635 - val_loss: 0.0279 - val_accuracy: 0.9941
Epoch 4/10
137/137 [==============================] - 131s 956ms/step - loss: 0.1359 - accuracy: 0.9680 - val_loss: 0.0360 - val_accuracy: 0.9934
Epoch 5/10
137/137 [==============================] - 128s 935ms/step - loss: 0.1344 - accuracy: 0.9654 - val_loss: 0.0384 - val_accuracy: 0.9934
Epoch 6/10
137/137 [==============================] - 128s 937ms/step - loss: 0.1269 - accuracy: 0.9688 - val_loss: 0.0345 - val_accuracy: 0.9944
Epoch 7/10
137/137 [==============================] - 128s 937ms/step - loss: 0.1056 - accuracy: 0.9721 - val_loss: 0.0238

In [56]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

5
Epoch 1/10
165/165 [==============================] - 159s 952ms/step - loss: 0.2742 - accuracy: 0.9392 - val_loss: 0.0305 - val_accuracy: 0.9929
Epoch 2/10
165/165 [==============================] - 158s 956ms/step - loss: 0.1704 - accuracy: 0.9554 - val_loss: 0.0260 - val_accuracy: 0.9929
Epoch 3/10
165/165 [==============================] - 157s 952ms/step - loss: 0.1541 - accuracy: 0.9617 - val_loss: 0.0260 - val_accuracy: 0.9931
Epoch 4/10
165/165 [==============================] - 156s 949ms/step - loss: 0.1309 - accuracy: 0.9668 - val_loss: 0.0371 - val_accuracy: 0.9941
Epoch 5/10
165/165 [==============================] - 156s 947ms/step - loss: 0.1315 - accuracy: 0.9670 - val_loss: 0.0235 - val_accuracy: 0.9957
Epoch 6/10
165/165 [==============================] - 157s 952ms/step - loss: 0.1223 - accuracy: 0.9699 - val_loss: 0.0277 - val_accuracy: 0.9936
Epoch 7/10
165/165 [==============================] - 156s 947ms/step - loss: 0.1146 - accuracy: 0.9698 - val_loss: 0.0391

In [57]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

6
Epoch 1/10
192/192 [==============================] - 184s 945ms/step - loss: 0.2182 - accuracy: 0.9497 - val_loss: 0.0506 - val_accuracy: 0.9918
Epoch 2/10
192/192 [==============================] - 182s 947ms/step - loss: 0.1566 - accuracy: 0.9602 - val_loss: 0.0258 - val_accuracy: 0.9949
Epoch 3/10
192/192 [==============================] - 182s 948ms/step - loss: 0.1316 - accuracy: 0.9654 - val_loss: 0.0274 - val_accuracy: 0.9934
Epoch 4/10
192/192 [==============================] - 182s 946ms/step - loss: 0.1381 - accuracy: 0.9651 - val_loss: 0.0448 - val_accuracy: 0.9890
Epoch 5/10
192/192 [==============================] - 181s 944ms/step - loss: 0.1342 - accuracy: 0.9635 - val_loss: 0.0281 - val_accuracy: 0.9941
Epoch 6/10
192/192 [==============================] - 181s 942ms/step - loss: 0.1367 - accuracy: 0.9658 - val_loss: 0.0499 - val_accuracy: 0.9890
Epoch 7/10
192/192 [==============================] - 183s 952ms/step - loss: 0.1185 - accuracy: 0.9694 - val_loss: 0.0316

In [58]:
n=7
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

7
Epoch 1/10
219/219 [==============================] - 207s 938ms/step - loss: 0.2189 - accuracy: 0.9519 - val_loss: 0.0276 - val_accuracy: 0.9944
Epoch 2/10
219/219 [==============================] - 197s 902ms/step - loss: 0.1577 - accuracy: 0.9603 - val_loss: 0.0624 - val_accuracy: 0.9903
Epoch 3/10
219/219 [==============================] - 198s 905ms/step - loss: 0.1410 - accuracy: 0.9642 - val_loss: 0.0417 - val_accuracy: 0.9949
Epoch 4/10
219/219 [==============================] - 198s 906ms/step - loss: 0.1175 - accuracy: 0.9703 - val_loss: 0.0301 - val_accuracy: 0.9952
Epoch 5/10
219/219 [==============================] - 198s 906ms/step - loss: 0.1348 - accuracy: 0.9655 - val_loss: 0.0402 - val_accuracy: 0.9916
Epoch 6/10
219/219 [==============================] - 198s 904ms/step - loss: 0.1327 - accuracy: 0.9667 - val_loss: 0.0440 - val_accuracy: 0.9936
Epoch 7/10
219/219 [==============================] - 198s 905ms/step - loss: 0.1516 - accuracy: 0.9662 - val_loss: 0.0253

In [59]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

8
Epoch 1/10
247/247 [==============================] - 233s 933ms/step - loss: 0.2246 - accuracy: 0.9511 - val_loss: 0.0328 - val_accuracy: 0.9913
Epoch 2/10
247/247 [==============================] - 230s 932ms/step - loss: 0.1463 - accuracy: 0.9633 - val_loss: 0.0282 - val_accuracy: 0.9941
Epoch 3/10
247/247 [==============================] - 231s 937ms/step - loss: 0.1514 - accuracy: 0.9614 - val_loss: 0.0415 - val_accuracy: 0.9903
Epoch 4/10
247/247 [==============================] - 234s 948ms/step - loss: 0.1305 - accuracy: 0.9666 - val_loss: 0.0328 - val_accuracy: 0.9949
Epoch 5/10
247/247 [==============================] - 232s 939ms/step - loss: 0.1283 - accuracy: 0.9678 - val_loss: 0.0282 - val_accuracy: 0.9946
Epoch 6/10
247/247 [==============================] - 233s 942ms/step - loss: 0.1214 - accuracy: 0.9677 - val_loss: 0.0365 - val_accuracy: 0.9936
Epoch 7/10
247/247 [==============================] - 233s 943ms/step - loss: 0.1075 - accuracy: 0.9723 - val_loss: 0.0525

In [60]:
n=9
print(n)#9

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

9
Epoch 1/10
274/274 [==============================] - 262s 949ms/step - loss: 0.2159 - accuracy: 0.9522 - val_loss: 0.0243 - val_accuracy: 0.9946
Epoch 2/10
274/274 [==============================] - 260s 950ms/step - loss: 0.1503 - accuracy: 0.9621 - val_loss: 0.0353 - val_accuracy: 0.9921
Epoch 3/10
274/274 [==============================] - 261s 952ms/step - loss: 0.1357 - accuracy: 0.9643 - val_loss: 0.0143 - val_accuracy: 0.9957
Epoch 4/10
274/274 [==============================] - 260s 949ms/step - loss: 0.1205 - accuracy: 0.9698 - val_loss: 0.0246 - val_accuracy: 0.9939
Epoch 5/10
274/274 [==============================] - 260s 950ms/step - loss: 0.1133 - accuracy: 0.9710 - val_loss: 0.0437 - val_accuracy: 0.9936
Epoch 6/10
274/274 [==============================] - 260s 948ms/step - loss: 0.1178 - accuracy: 0.9693 - val_loss: 0.0358 - val_accuracy: 0.9918
Epoch 7/10
274/274 [==============================] - 260s 950ms/step - loss: 0.1309 - accuracy: 0.9671 - val_loss: 0.0214

In [61]:
n=10
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

10
Epoch 1/10
301/301 [==============================] - 299s 987ms/step - loss: 0.2009 - accuracy: 0.9519 - val_loss: 0.0300 - val_accuracy: 0.9946
Epoch 2/10
301/301 [==============================] - 282s 937ms/step - loss: 0.1483 - accuracy: 0.9615 - val_loss: 0.0663 - val_accuracy: 0.9855
Epoch 3/10
301/301 [==============================] - 282s 939ms/step - loss: 0.1423 - accuracy: 0.9624 - val_loss: 0.0365 - val_accuracy: 0.9916
Epoch 4/10
301/301 [==============================] - 294s 978ms/step - loss: 0.1362 - accuracy: 0.9642 - val_loss: 0.0416 - val_accuracy: 0.9931
Epoch 5/10
301/301 [==============================] - 296s 982ms/step - loss: 0.1252 - accuracy: 0.9683 - val_loss: 0.0291 - val_accuracy: 0.9944
Epoch 6/10
301/301 [==============================] - 296s 983ms/step - loss: 0.1201 - accuracy: 0.9695 - val_loss: 0.0327 - val_accuracy: 0.9923
Epoch 7/10
301/301 [==============================] - 295s 981ms/step - loss: 0.1107 - accuracy: 0.9725 - val_loss: 0.040

In [62]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

11
Epoch 1/10
329/329 [==============================] - 327s 987ms/step - loss: 0.2297 - accuracy: 0.9485 - val_loss: 0.0495 - val_accuracy: 0.9921
Epoch 2/10
329/329 [==============================] - 322s 979ms/step - loss: 0.1623 - accuracy: 0.9585 - val_loss: 0.0921 - val_accuracy: 0.9885
Epoch 3/10
329/329 [==============================] - 321s 977ms/step - loss: 0.1342 - accuracy: 0.9668 - val_loss: 0.0593 - val_accuracy: 0.9903
Epoch 4/10
329/329 [==============================] - 322s 979ms/step - loss: 0.1367 - accuracy: 0.9668 - val_loss: 0.0315 - val_accuracy: 0.9944
Epoch 5/10
329/329 [==============================] - 322s 980ms/step - loss: 0.1325 - accuracy: 0.9685 - val_loss: 0.0380 - val_accuracy: 0.9934
Epoch 6/10
329/329 [==============================] - 322s 978ms/step - loss: 0.1231 - accuracy: 0.9714 - val_loss: 0.1756 - val_accuracy: 0.9622
Epoch 7/10
329/329 [==============================] - 321s 977ms/step - loss: 0.1314 - accuracy: 0.9680 - val_loss: 0.053

In [63]:
n=12
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

12
Epoch 1/10
356/356 [==============================] - 350s 976ms/step - loss: 0.2028 - accuracy: 0.9546 - val_loss: 0.0321 - val_accuracy: 0.9929
Epoch 2/10
356/356 [==============================] - 349s 980ms/step - loss: 0.1341 - accuracy: 0.9639 - val_loss: 0.0299 - val_accuracy: 0.9934
Epoch 3/10
356/356 [==============================] - 347s 974ms/step - loss: 0.1341 - accuracy: 0.9650 - val_loss: 0.0316 - val_accuracy: 0.9918
Epoch 4/10
356/356 [==============================] - 347s 974ms/step - loss: 0.1200 - accuracy: 0.9694 - val_loss: 0.0749 - val_accuracy: 0.9903
Epoch 5/10
356/356 [==============================] - 347s 975ms/step - loss: 0.1255 - accuracy: 0.9691 - val_loss: 0.0376 - val_accuracy: 0.9939
Epoch 6/10
356/356 [==============================] - 347s 974ms/step - loss: 0.1373 - accuracy: 0.9688 - val_loss: 0.0517 - val_accuracy: 0.9898
Epoch 7/10
356/356 [==============================] - 347s 976ms/step - loss: 0.1296 - accuracy: 0.9696 - val_loss: 0.018

In [64]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

13
Epoch 1/10
383/383 [==============================] - 378s 982ms/step - loss: 0.2005 - accuracy: 0.9542 - val_loss: 0.0217 - val_accuracy: 0.9954
Epoch 2/10
383/383 [==============================] - 378s 987ms/step - loss: 0.1455 - accuracy: 0.9622 - val_loss: 0.0206 - val_accuracy: 0.9954
Epoch 3/10
383/383 [==============================] - 377s 984ms/step - loss: 0.1406 - accuracy: 0.9647 - val_loss: 0.0224 - val_accuracy: 0.9952
Epoch 4/10
383/383 [==============================] - 376s 981ms/step - loss: 0.1287 - accuracy: 0.9679 - val_loss: 0.0326 - val_accuracy: 0.9934
Epoch 5/10
383/383 [==============================] - 376s 982ms/step - loss: 0.1285 - accuracy: 0.9694 - val_loss: 0.0373 - val_accuracy: 0.9929
Epoch 6/10
383/383 [==============================] - 377s 984ms/step - loss: 0.1169 - accuracy: 0.9700 - val_loss: 0.0161 - val_accuracy: 0.9959
Epoch 7/10
383/383 [==============================] - 376s 981ms/step - loss: 0.1346 - accuracy: 0.9687 - val_loss: 0.036

In [65]:
print(n)#14

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

14
Epoch 1/10
411/411 [==============================] - 409s 989ms/step - loss: 0.2093 - accuracy: 0.9524 - val_loss: 0.0197 - val_accuracy: 0.9952
Epoch 2/10
411/411 [==============================] - 414s 1s/step - loss: 0.1478 - accuracy: 0.9636 - val_loss: 0.0185 - val_accuracy: 0.9952
Epoch 3/10
411/411 [==============================] - 411s 999ms/step - loss: 0.1367 - accuracy: 0.9659 - val_loss: 0.0496 - val_accuracy: 0.9936
Epoch 4/10
411/411 [==============================] - 410s 997ms/step - loss: 0.1292 - accuracy: 0.9683 - val_loss: 0.0325 - val_accuracy: 0.9921
Epoch 5/10
411/411 [==============================] - 409s 996ms/step - loss: 0.1253 - accuracy: 0.9679 - val_loss: 0.0393 - val_accuracy: 0.9923
Epoch 6/10
411/411 [==============================] - 410s 998ms/step - loss: 0.1222 - accuracy: 0.9698 - val_loss: 0.0830 - val_accuracy: 0.9788
Epoch 7/10
411/411 [==============================] - 409s 995ms/step - loss: 0.1343 - accuracy: 0.9699 - val_loss: 0.0196 -

In [66]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

15
Epoch 1/10
438/438 [==============================] - 446s 1s/step - loss: 0.2095 - accuracy: 0.9569 - val_loss: 0.0197 - val_accuracy: 0.9952
Epoch 2/10
438/438 [==============================] - 424s 967ms/step - loss: 0.1464 - accuracy: 0.9637 - val_loss: 0.0258 - val_accuracy: 0.9949
Epoch 3/10
438/438 [==============================] - 416s 950ms/step - loss: 0.1333 - accuracy: 0.9672 - val_loss: 0.0370 - val_accuracy: 0.9944
Epoch 4/10
438/438 [==============================] - 416s 949ms/step - loss: 0.1273 - accuracy: 0.9675 - val_loss: 0.0218 - val_accuracy: 0.9954
Epoch 5/10
438/438 [==============================] - 399s 911ms/step - loss: 0.1320 - accuracy: 0.9672 - val_loss: 0.0328 - val_accuracy: 0.9944
Epoch 6/10
438/438 [==============================] - 440s 1s/step - loss: 0.1359 - accuracy: 0.9687 - val_loss: 0.0236 - val_accuracy: 0.9957
Epoch 7/10
438/438 [==============================] - 445s 1s/step - loss: 0.1258 - accuracy: 0.9706 - val_loss: 0.0257 - val_a

In [67]:
n=16
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

16
Epoch 1/10
465/465 [==============================] - 475s 1s/step - loss: 0.1864 - accuracy: 0.9553 - val_loss: 0.0286 - val_accuracy: 0.9944
Epoch 2/10
465/465 [==============================] - 474s 1s/step - loss: 0.1385 - accuracy: 0.9643 - val_loss: 0.0307 - val_accuracy: 0.9931
Epoch 3/10
465/465 [==============================] - 472s 1s/step - loss: 0.1372 - accuracy: 0.9660 - val_loss: 0.0368 - val_accuracy: 0.9911
Epoch 4/10
465/465 [==============================] - 473s 1s/step - loss: 0.1323 - accuracy: 0.9688 - val_loss: 0.0421 - val_accuracy: 0.9918
Epoch 5/10
465/465 [==============================] - 473s 1s/step - loss: 0.1282 - accuracy: 0.9688 - val_loss: 0.0640 - val_accuracy: 0.9885
Epoch 6/10
465/465 [==============================] - 476s 1s/step - loss: 0.1359 - accuracy: 0.9684 - val_loss: 0.0232 - val_accuracy: 0.9952
Epoch 7/10
465/465 [==============================] - 474s 1s/step - loss: 0.1260 - accuracy: 0.9704 - val_loss: 0.0228 - val_accuracy: 0.9

In [68]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

17
Epoch 1/10
493/493 [==============================] - 458s 924ms/step - loss: 0.1853 - accuracy: 0.9567 - val_loss: 0.0580 - val_accuracy: 0.9844
Epoch 2/10
493/493 [==============================] - 497s 1s/step - loss: 0.1412 - accuracy: 0.9641 - val_loss: 0.0196 - val_accuracy: 0.9962
Epoch 3/10
493/493 [==============================] - 506s 1s/step - loss: 0.1301 - accuracy: 0.9665 - val_loss: 0.0297 - val_accuracy: 0.9939
Epoch 4/10
493/493 [==============================] - 508s 1s/step - loss: 0.1319 - accuracy: 0.9691 - val_loss: 0.0255 - val_accuracy: 0.9952
Epoch 5/10
493/493 [==============================] - 505s 1s/step - loss: 0.1365 - accuracy: 0.9678 - val_loss: 0.0516 - val_accuracy: 0.9885
Epoch 6/10
493/493 [==============================] - 506s 1s/step - loss: 0.1253 - accuracy: 0.9707 - val_loss: 0.0253 - val_accuracy: 0.9949
Epoch 7/10
493/493 [==============================] - 506s 1s/step - loss: 0.1291 - accuracy: 0.9695 - val_loss: 0.0400 - val_accuracy: 

In [69]:
n=18
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

18
Epoch 1/10
520/520 [==============================] - 543s 1s/step - loss: 0.1924 - accuracy: 0.9548 - val_loss: 0.0288 - val_accuracy: 0.9929
Epoch 2/10
520/520 [==============================] - 541s 1s/step - loss: 0.1343 - accuracy: 0.9660 - val_loss: 0.0205 - val_accuracy: 0.9962
Epoch 3/10
520/520 [==============================] - 540s 1s/step - loss: 0.1303 - accuracy: 0.9670 - val_loss: 0.0226 - val_accuracy: 0.9952
Epoch 4/10
520/520 [==============================] - 541s 1s/step - loss: 0.1280 - accuracy: 0.9673 - val_loss: 0.0334 - val_accuracy: 0.9941
Epoch 5/10
520/520 [==============================] - 540s 1s/step - loss: 0.1246 - accuracy: 0.9690 - val_loss: 0.0488 - val_accuracy: 0.9893
Epoch 6/10
520/520 [==============================] - 537s 1s/step - loss: 0.1237 - accuracy: 0.9709 - val_loss: 0.0263 - val_accuracy: 0.9941
Epoch 7/10
520/520 [==============================] - 488s 939ms/step - loss: 0.1353 - accuracy: 0.9696 - val_loss: 0.0200 - val_accuracy: 

In [70]:
n=19
print(n)#19

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

19
Epoch 1/10
547/547 [==============================] - 559s 1s/step - loss: 0.1834 - accuracy: 0.9562 - val_loss: 0.0299 - val_accuracy: 0.9934
Epoch 2/10
547/547 [==============================] - 563s 1s/step - loss: 0.1409 - accuracy: 0.9644 - val_loss: 0.0221 - val_accuracy: 0.9946
Epoch 3/10
547/547 [==============================] - 563s 1s/step - loss: 0.1273 - accuracy: 0.9672 - val_loss: 0.0478 - val_accuracy: 0.9936
Epoch 4/10
547/547 [==============================] - 563s 1s/step - loss: 0.1274 - accuracy: 0.9693 - val_loss: 0.0305 - val_accuracy: 0.9949
Epoch 5/10
547/547 [==============================] - 552s 1s/step - loss: 0.1329 - accuracy: 0.9713 - val_loss: 0.0355 - val_accuracy: 0.9923
Epoch 6/10
547/547 [==============================] - 505s 923ms/step - loss: 0.1357 - accuracy: 0.9683 - val_loss: 0.0290 - val_accuracy: 0.9931
Epoch 7/10
547/547 [==============================] - 504s 922ms/step - loss: 0.1444 - accuracy: 0.9671 - val_loss: 0.0345 - val_accurac

In [71]:

print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

20
Epoch 1/10
552/552 [==============================] - 569s 1s/step - loss: 0.2015 - accuracy: 0.9549 - val_loss: 0.0425 - val_accuracy: 0.9911
Epoch 2/10
552/552 [==============================] - 571s 1s/step - loss: 0.1350 - accuracy: 0.9655 - val_loss: 0.0230 - val_accuracy: 0.9941
Epoch 3/10
552/552 [==============================] - 573s 1s/step - loss: 0.1430 - accuracy: 0.9661 - val_loss: 0.0202 - val_accuracy: 0.9946
Epoch 4/10
552/552 [==============================] - 571s 1s/step - loss: 0.1333 - accuracy: 0.9682 - val_loss: 0.0241 - val_accuracy: 0.9946
Epoch 5/10
552/552 [==============================] - 569s 1s/step - loss: 0.1241 - accuracy: 0.9706 - val_loss: 0.0230 - val_accuracy: 0.9918
Epoch 6/10
552/552 [==============================] - 569s 1s/step - loss: 0.1309 - accuracy: 0.9685 - val_loss: 0.0293 - val_accuracy: 0.9949
Epoch 7/10
552/552 [==============================] - 538s 975ms/step - loss: 0.1248 - accuracy: 0.9706 - val_loss: 0.0322 - val_accuracy: 

In [72]:
new_model_random_dir  = "D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_random_e1"

i=0

for model in models_random:
    model.save(new_model_random_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_random_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_random_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_random_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_random_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_random_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_random_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_random_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_random_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/mo

In [ ]:
loading = False

models_random = []

if loading:
    for i in range(20):
        model_random_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_random_e1_"+str(i)
        print(model_random_dir)
        model =utils.My_model('gtsrb',True,model_random_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_random.append(model)
       

In [73]:
try:
    del random_values
    del top_images_by_random
    del top_labels_by_random
    del image_sets_random
    del label_sets_random
    del models_random
except:
    print("Error")

Error


In [74]:
gc.collect()

72486

## Training guided by NC

In [16]:
# NC
nc_values = []
for i in range(1,13):
    #save_dir = "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/NC_values/nc_values_"+str(i)+".npy"
    save_dir = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/gtsrb_nc_values_"+str(i)+".npy"

    #print(save_dir_rand)
    tmp_values = np.load(save_dir)
    #print(tmp_values.shape)
    nc_values = np.append(nc_values,tmp_values)

In [17]:
nc_values.shape

(35287,)

In [18]:

top_images_by_nc = utils.get_x_of_indexes(list(np.flip(np.argsort(nc_values))),x_train_and_adversary)
top_labels_by_nc = utils.get_x_of_indexes(list(np.flip(np.argsort(nc_values))),y_train_and_adversary)


In [19]:
m = n_data_points
n = 0
image_sets_nc = []
label_sets_nc = []


for i in range((len(top_images_by_nc)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_nc)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_nc)%m))
        top_images_by_nc_n = np.array(top_images_by_nc[:n+m+(len(top_images_by_nc)%m)])
        top_labels_by_nc_n = np.array(top_labels_by_nc[:n+m+(len(top_images_by_nc)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_nc_n = np.array(top_images_by_nc[:n+m])
        top_labels_by_nc_n = np.array(top_labels_by_nc[:n+m])
    image_sets_nc.append(top_images_by_nc_n)
    label_sets_nc.append(top_labels_by_nc_n)
    print(len(top_images_by_nc_n))
    n += m




0 :
0  ->  1750
1750
1 :
0  ->  3500
3500
2 :
0  ->  5250
5250
3 :
0  ->  7000
7000
4 :
0  ->  8750
8750
5 :
0  ->  10500
10500
6 :
0  ->  12250
12250
7 :
0  ->  14000
14000
8 :
0  ->  15750
15750
9 :
0  ->  17500
17500
10 :
0  ->  19250
19250
11 :
0  ->  21000
21000
12 :
0  ->  22750
22750
13 :
0  ->  24500
24500
14 :
0  ->  26250
26250
15 :
0  ->  28000
28000
16 :
0  ->  29750
29750
17 :
0  ->  31500
31500
18 :
0  ->  33250
33250
19 :
0  ->  35000
35000
20 :
Last
0  ->  37037
35287


In [20]:
print(model_dir)

models_nc = []
for i in range(len(label_sets_nc)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_nc.append(model)


C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06/
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled
20 :
Model loaded correctly
Model compiled


In [21]:
n=0
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

0
Epoch 1/10
28/28 [==============================] - 33s 584ms/step - loss: 0.1168 - accuracy: 0.9818 - val_loss: 0.1387 - val_accuracy: 0.9671
Epoch 2/10
28/28 [==============================] - 16s 565ms/step - loss: 0.0337 - accuracy: 0.9918 - val_loss: 0.7072 - val_accuracy: 0.8686
Epoch 3/10
28/28 [==============================] - 16s 573ms/step - loss: 0.0272 - accuracy: 0.9922 - val_loss: 0.6684 - val_accuracy: 0.8798
Epoch 4/10
28/28 [==============================] - 16s 569ms/step - loss: 0.0114 - accuracy: 0.9976 - val_loss: 1.5312 - val_accuracy: 0.8701
Epoch 5/10
28/28 [==============================] - 16s 571ms/step - loss: 0.0217 - accuracy: 0.9944 - val_loss: 1.2311 - val_accuracy: 0.8569
Epoch 6/10
28/28 [==============================] - 16s 587ms/step - loss: 0.0186 - accuracy: 0.9933 - val_loss: 1.5154 - val_accuracy: 0.8847
Epoch 7/10
28/28 [==============================] - 16s 578ms/step - loss: 0.0116 - accuracy: 0.9973 - val_loss: 1.1641 - val_accuracy: 0.89

In [22]:

print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

1
Epoch 1/10
55/55 [==============================] - 30s 506ms/step - loss: 0.1594 - accuracy: 0.9884 - val_loss: 0.1494 - val_accuracy: 0.9594
Epoch 2/10
55/55 [==============================] - 28s 503ms/step - loss: 0.0238 - accuracy: 0.9951 - val_loss: 0.1331 - val_accuracy: 0.9668
Epoch 3/10
55/55 [==============================] - 30s 539ms/step - loss: 0.0334 - accuracy: 0.9927 - val_loss: 0.2034 - val_accuracy: 0.9456
Epoch 4/10
55/55 [==============================] - 30s 554ms/step - loss: 0.0099 - accuracy: 0.9974 - val_loss: 0.7390 - val_accuracy: 0.7908
Epoch 5/10
55/55 [==============================] - 29s 524ms/step - loss: 0.0301 - accuracy: 0.9928 - val_loss: 2.2895 - val_accuracy: 0.6680
Epoch 6/10
55/55 [==============================] - 28s 509ms/step - loss: 0.0247 - accuracy: 0.9953 - val_loss: 1.6315 - val_accuracy: 0.6591
Epoch 7/10
55/55 [==============================] - 28s 509ms/step - loss: 0.0196 - accuracy: 0.9949 - val_loss: 2.0996 - val_accuracy: 0.70

In [23]:

print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

2
Epoch 1/10
83/83 [==============================] - 41s 477ms/step - loss: 0.1362 - accuracy: 0.9761 - val_loss: 0.2078 - val_accuracy: 0.9398
Epoch 2/10
83/83 [==============================] - 39s 476ms/step - loss: 0.0682 - accuracy: 0.9803 - val_loss: 0.1356 - val_accuracy: 0.9701
Epoch 3/10
83/83 [==============================] - 40s 479ms/step - loss: 0.0431 - accuracy: 0.9882 - val_loss: 0.9537 - val_accuracy: 0.8811
Epoch 4/10
83/83 [==============================] - 39s 472ms/step - loss: 0.0625 - accuracy: 0.9849 - val_loss: 0.4695 - val_accuracy: 0.9148
Epoch 5/10
83/83 [==============================] - 40s 478ms/step - loss: 0.0483 - accuracy: 0.9879 - val_loss: 1.7273 - val_accuracy: 0.8094
Epoch 6/10
83/83 [==============================] - 39s 471ms/step - loss: 0.0525 - accuracy: 0.9887 - val_loss: 1.7611 - val_accuracy: 0.7811
Epoch 7/10
83/83 [==============================] - 39s 471ms/step - loss: 0.0671 - accuracy: 0.9903 - val_loss: 2.1364 - val_accuracy: 0.75

In [24]:

print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

3
Epoch 1/10
110/110 [==============================] - 53s 463ms/step - loss: 0.1455 - accuracy: 0.9765 - val_loss: 0.0509 - val_accuracy: 0.9880
Epoch 2/10
110/110 [==============================] - 50s 458ms/step - loss: 0.0778 - accuracy: 0.9808 - val_loss: 0.1755 - val_accuracy: 0.9742
Epoch 3/10
110/110 [==============================] - 51s 460ms/step - loss: 0.0730 - accuracy: 0.9813 - val_loss: 0.2070 - val_accuracy: 0.9627
Epoch 4/10
110/110 [==============================] - 51s 459ms/step - loss: 0.0585 - accuracy: 0.9822 - val_loss: 0.2774 - val_accuracy: 0.9444
Epoch 5/10
110/110 [==============================] - 51s 464ms/step - loss: 0.0533 - accuracy: 0.9848 - val_loss: 0.9754 - val_accuracy: 0.8303
Epoch 6/10
110/110 [==============================] - 50s 459ms/step - loss: 0.0482 - accuracy: 0.9873 - val_loss: 1.3860 - val_accuracy: 0.8188
Epoch 7/10
110/110 [==============================] - 50s 459ms/step - loss: 0.0509 - accuracy: 0.9878 - val_loss: 2.7548 - val_

In [25]:
#4
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

4
Epoch 1/10
137/137 [==============================] - 64s 453ms/step - loss: 0.1903 - accuracy: 0.9717 - val_loss: 0.0693 - val_accuracy: 0.9862
Epoch 2/10
137/137 [==============================] - 61s 449ms/step - loss: 0.0903 - accuracy: 0.9809 - val_loss: 0.0846 - val_accuracy: 0.9798
Epoch 3/10
137/137 [==============================] - 65s 477ms/step - loss: 0.0829 - accuracy: 0.9809 - val_loss: 0.8605 - val_accuracy: 0.8566
Epoch 4/10
137/137 [==============================] - 68s 499ms/step - loss: 0.0668 - accuracy: 0.9832 - val_loss: 0.5265 - val_accuracy: 0.9023
Epoch 5/10
137/137 [==============================] - 69s 504ms/step - loss: 0.0653 - accuracy: 0.9835 - val_loss: 1.3465 - val_accuracy: 0.7821
Epoch 6/10
137/137 [==============================] - 67s 490ms/step - loss: 0.0733 - accuracy: 0.9832 - val_loss: 2.5920 - val_accuracy: 0.7071
Epoch 7/10
137/137 [==============================] - 67s 492ms/step - loss: 0.0650 - accuracy: 0.9840 - val_loss: 3.2775 - val_

In [26]:
n=5
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

5
Epoch 1/10
165/165 [==============================] - 82s 485ms/step - loss: 0.1405 - accuracy: 0.9747 - val_loss: 0.0586 - val_accuracy: 0.9872
Epoch 2/10
165/165 [==============================] - 80s 483ms/step - loss: 0.0777 - accuracy: 0.9811 - val_loss: 0.1047 - val_accuracy: 0.9724
Epoch 3/10
165/165 [==============================] - 79s 480ms/step - loss: 0.0690 - accuracy: 0.9829 - val_loss: 0.8015 - val_accuracy: 0.9010
Epoch 4/10
165/165 [==============================] - 79s 482ms/step - loss: 0.0825 - accuracy: 0.9810 - val_loss: 1.5185 - val_accuracy: 0.8357
Epoch 5/10
165/165 [==============================] - 79s 480ms/step - loss: 0.0667 - accuracy: 0.9852 - val_loss: 1.8744 - val_accuracy: 0.6923
Epoch 6/10
165/165 [==============================] - 79s 480ms/step - loss: 0.0773 - accuracy: 0.9842 - val_loss: 1.8461 - val_accuracy: 0.6522
Epoch 7/10
165/165 [==============================] - 79s 480ms/step - loss: 0.0753 - accuracy: 0.9837 - val_loss: 6.5990 - val_

In [27]:
n=6
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

6
Epoch 1/10
192/192 [==============================] - 95s 484ms/step - loss: 0.1197 - accuracy: 0.9751 - val_loss: 0.0858 - val_accuracy: 0.9788
Epoch 2/10
192/192 [==============================] - 92s 477ms/step - loss: 0.0690 - accuracy: 0.9827 - val_loss: 0.4155 - val_accuracy: 0.9265
Epoch 3/10
192/192 [==============================] - 92s 480ms/step - loss: 0.0651 - accuracy: 0.9855 - val_loss: 0.7991 - val_accuracy: 0.8783
Epoch 4/10
192/192 [==============================] - 91s 476ms/step - loss: 0.0735 - accuracy: 0.9831 - val_loss: 1.8181 - val_accuracy: 0.7854
Epoch 5/10
192/192 [==============================] - 92s 477ms/step - loss: 0.0679 - accuracy: 0.9842 - val_loss: 2.9304 - val_accuracy: 0.7395
Epoch 6/10
192/192 [==============================] - 92s 478ms/step - loss: 0.0596 - accuracy: 0.9863 - val_loss: 6.3739 - val_accuracy: 0.7022
Epoch 7/10
192/192 [==============================] - 91s 475ms/step - loss: 0.0809 - accuracy: 0.9845 - val_loss: 4.9242 - val_

In [28]:
n=7
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

7
Epoch 1/10
219/219 [==============================] - 107s 478ms/step - loss: 0.1186 - accuracy: 0.9779 - val_loss: 0.1294 - val_accuracy: 0.9781
Epoch 2/10
219/219 [==============================] - 105s 482ms/step - loss: 0.0772 - accuracy: 0.9829 - val_loss: 0.4263 - val_accuracy: 0.9199
Epoch 3/10
219/219 [==============================] - 104s 476ms/step - loss: 0.0683 - accuracy: 0.9848 - val_loss: 1.1581 - val_accuracy: 0.8359
Epoch 4/10
219/219 [==============================] - 104s 473ms/step - loss: 0.0726 - accuracy: 0.9839 - val_loss: 2.5522 - val_accuracy: 0.7385
Epoch 5/10
219/219 [==============================] - 104s 473ms/step - loss: 0.0649 - accuracy: 0.9863 - val_loss: 4.6058 - val_accuracy: 0.6971
Epoch 6/10
219/219 [==============================] - 104s 473ms/step - loss: 0.0620 - accuracy: 0.9861 - val_loss: 3.8872 - val_accuracy: 0.6762
Epoch 7/10
219/219 [==============================] - 103s 471ms/step - loss: 0.0759 - accuracy: 0.9841 - val_loss: 7.4338

In [29]:
n=8
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

8
Epoch 1/10
247/247 [==============================] - 119s 473ms/step - loss: 0.1246 - accuracy: 0.9749 - val_loss: 0.0869 - val_accuracy: 0.9829
Epoch 2/10
247/247 [==============================] - 116s 469ms/step - loss: 0.0821 - accuracy: 0.9825 - val_loss: 0.4162 - val_accuracy: 0.9428
Epoch 3/10
247/247 [==============================] - 116s 470ms/step - loss: 0.0765 - accuracy: 0.9824 - val_loss: 0.7164 - val_accuracy: 0.8643
Epoch 4/10
247/247 [==============================] - 116s 469ms/step - loss: 0.0742 - accuracy: 0.9805 - val_loss: 1.8596 - val_accuracy: 0.7890
Epoch 5/10
247/247 [==============================] - 119s 480ms/step - loss: 0.0754 - accuracy: 0.9818 - val_loss: 3.7748 - val_accuracy: 0.7604
Epoch 6/10
247/247 [==============================] - 117s 474ms/step - loss: 0.0761 - accuracy: 0.9843 - val_loss: 4.4904 - val_accuracy: 0.7464
Epoch 7/10
247/247 [==============================] - 116s 471ms/step - loss: 0.0734 - accuracy: 0.9835 - val_loss: 6.4160

In [30]:
#reiniciar la 9
print(n)#9

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

9
Epoch 1/10
274/274 [==============================] - 131s 471ms/step - loss: 0.1226 - accuracy: 0.9752 - val_loss: 0.1610 - val_accuracy: 0.9668
Epoch 2/10
274/274 [==============================] - 128s 467ms/step - loss: 0.0731 - accuracy: 0.9802 - val_loss: 0.3076 - val_accuracy: 0.9423
Epoch 3/10
274/274 [==============================] - 128s 469ms/step - loss: 0.0710 - accuracy: 0.9831 - val_loss: 1.1499 - val_accuracy: 0.8699
Epoch 4/10
274/274 [==============================] - 129s 471ms/step - loss: 0.0727 - accuracy: 0.9840 - val_loss: 1.8714 - val_accuracy: 0.8357
Epoch 5/10
274/274 [==============================] - 129s 470ms/step - loss: 0.0819 - accuracy: 0.9843 - val_loss: 1.9405 - val_accuracy: 0.8214
Epoch 6/10
274/274 [==============================] - 128s 469ms/step - loss: 0.0775 - accuracy: 0.9841 - val_loss: 3.4105 - val_accuracy: 0.8094
Epoch 7/10
274/274 [==============================] - 128s 468ms/step - loss: 0.0833 - accuracy: 0.9838 - val_loss: 6.2936

In [31]:
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

10
Epoch 1/10
301/301 [==============================] - 144s 471ms/step - loss: 0.1399 - accuracy: 0.9715 - val_loss: 0.1392 - val_accuracy: 0.9763
Epoch 2/10
301/301 [==============================] - 141s 468ms/step - loss: 0.0944 - accuracy: 0.9798 - val_loss: 0.5823 - val_accuracy: 0.9319
Epoch 3/10
301/301 [==============================] - 141s 468ms/step - loss: 0.0850 - accuracy: 0.9812 - val_loss: 0.7827 - val_accuracy: 0.8556
Epoch 4/10
301/301 [==============================] - 140s 466ms/step - loss: 0.0849 - accuracy: 0.9816 - val_loss: 1.9645 - val_accuracy: 0.8492
Epoch 5/10
301/301 [==============================] - 140s 466ms/step - loss: 0.0815 - accuracy: 0.9829 - val_loss: 1.3061 - val_accuracy: 0.7928
Epoch 6/10
301/301 [==============================] - 141s 467ms/step - loss: 0.0833 - accuracy: 0.9814 - val_loss: 2.0096 - val_accuracy: 0.8181
Epoch 7/10
301/301 [==============================] - 140s 466ms/step - loss: 0.0790 - accuracy: 0.9837 - val_loss: 4.907

In [32]:
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

11
Epoch 1/10
329/329 [==============================] - 156s 467ms/step - loss: 0.1090 - accuracy: 0.9773 - val_loss: 0.1147 - val_accuracy: 0.9758
Epoch 2/10
329/329 [==============================] - 153s 465ms/step - loss: 0.0829 - accuracy: 0.9810 - val_loss: 0.4008 - val_accuracy: 0.9224
Epoch 3/10
329/329 [==============================] - 153s 465ms/step - loss: 0.0833 - accuracy: 0.9795 - val_loss: 1.3408 - val_accuracy: 0.8663
Epoch 4/10
329/329 [==============================] - 155s 472ms/step - loss: 0.0826 - accuracy: 0.9812 - val_loss: 2.7988 - val_accuracy: 0.8392
Epoch 5/10
329/329 [==============================] - 153s 464ms/step - loss: 0.0690 - accuracy: 0.9829 - val_loss: 3.6488 - val_accuracy: 0.8367
Epoch 6/10
329/329 [==============================] - 153s 466ms/step - loss: 0.0789 - accuracy: 0.9831 - val_loss: 5.1532 - val_accuracy: 0.8288
Epoch 7/10
329/329 [==============================] - 152s 463ms/step - loss: 0.1034 - accuracy: 0.9816 - val_loss: 3.245

In [33]:
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

12
Epoch 1/10
356/356 [==============================] - 168s 466ms/step - loss: 0.1173 - accuracy: 0.9747 - val_loss: 0.0851 - val_accuracy: 0.9839
Epoch 2/10
356/356 [==============================] - 165s 464ms/step - loss: 0.0771 - accuracy: 0.9816 - val_loss: 0.2779 - val_accuracy: 0.9393
Epoch 3/10
356/356 [==============================] - 165s 462ms/step - loss: 0.0801 - accuracy: 0.9816 - val_loss: 0.7465 - val_accuracy: 0.8423
Epoch 4/10
356/356 [==============================] - 164s 461ms/step - loss: 0.0784 - accuracy: 0.9807 - val_loss: 2.5046 - val_accuracy: 0.8543
Epoch 5/10
356/356 [==============================] - 165s 465ms/step - loss: 0.0900 - accuracy: 0.9812 - val_loss: 2.3751 - val_accuracy: 0.8497
Epoch 6/10
356/356 [==============================] - 156s 439ms/step - loss: 0.0843 - accuracy: 0.9806 - val_loss: 2.9826 - val_accuracy: 0.8487
Epoch 7/10
356/356 [==============================] - 151s 423ms/step - loss: 0.0970 - accuracy: 0.9821 - val_loss: 5.324

In [34]:
n=13
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

13
Epoch 1/10
383/383 [==============================] - 164s 424ms/step - loss: 0.1494 - accuracy: 0.9714 - val_loss: 0.1171 - val_accuracy: 0.9801
Epoch 2/10
383/383 [==============================] - 162s 422ms/step - loss: 0.0830 - accuracy: 0.9796 - val_loss: 0.4027 - val_accuracy: 0.9446
Epoch 3/10
383/383 [==============================] - 162s 422ms/step - loss: 0.0886 - accuracy: 0.9807 - val_loss: 0.7865 - val_accuracy: 0.8921
Epoch 4/10
383/383 [==============================] - 161s 421ms/step - loss: 0.0788 - accuracy: 0.9806 - val_loss: 2.0650 - val_accuracy: 0.8739
Epoch 5/10
383/383 [==============================] - 166s 433ms/step - loss: 0.0945 - accuracy: 0.9808 - val_loss: 2.4872 - val_accuracy: 0.8696
Epoch 6/10
383/383 [==============================] - 177s 461ms/step - loss: 0.0931 - accuracy: 0.9804 - val_loss: 4.1182 - val_accuracy: 0.8614
Epoch 7/10
383/383 [==============================] - 177s 463ms/step - loss: 0.0905 - accuracy: 0.9808 - val_loss: 3.225

In [35]:
#14
n=14
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

14
Epoch 1/10
411/411 [==============================] - 195s 468ms/step - loss: 0.1290 - accuracy: 0.9736 - val_loss: 0.0685 - val_accuracy: 0.9839
Epoch 2/10
411/411 [==============================] - 189s 460ms/step - loss: 0.0977 - accuracy: 0.9773 - val_loss: 0.6190 - val_accuracy: 0.9291
Epoch 3/10
411/411 [==============================] - 189s 459ms/step - loss: 0.0856 - accuracy: 0.9789 - val_loss: 1.2251 - val_accuracy: 0.9058
Epoch 4/10
411/411 [==============================] - 190s 463ms/step - loss: 0.1022 - accuracy: 0.9782 - val_loss: 1.6882 - val_accuracy: 0.8997
Epoch 5/10
411/411 [==============================] - 189s 459ms/step - loss: 0.1066 - accuracy: 0.9777 - val_loss: 2.4947 - val_accuracy: 0.8913
Epoch 6/10
411/411 [==============================] - 190s 461ms/step - loss: 0.0919 - accuracy: 0.9809 - val_loss: 2.7537 - val_accuracy: 0.8887
Epoch 7/10
411/411 [==============================] - 189s 459ms/step - loss: 0.1044 - accuracy: 0.9804 - val_loss: 2.313

In [36]:
time.sleep(180)

In [37]:

print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

15
Epoch 1/10
438/438 [==============================] - 197s 444ms/step - loss: 0.1238 - accuracy: 0.9732 - val_loss: 0.0543 - val_accuracy: 0.9857
Epoch 2/10
438/438 [==============================] - 204s 466ms/step - loss: 0.0937 - accuracy: 0.9776 - val_loss: 0.4364 - val_accuracy: 0.9497
Epoch 3/10
438/438 [==============================] - 204s 465ms/step - loss: 0.0978 - accuracy: 0.9776 - val_loss: 0.7659 - val_accuracy: 0.9291
Epoch 4/10
438/438 [==============================] - 204s 467ms/step - loss: 0.0963 - accuracy: 0.9783 - val_loss: 1.2623 - val_accuracy: 0.9242
Epoch 5/10
438/438 [==============================] - 207s 473ms/step - loss: 0.1008 - accuracy: 0.9791 - val_loss: 1.2328 - val_accuracy: 0.9150
Epoch 6/10
438/438 [==============================] - 204s 465ms/step - loss: 0.1028 - accuracy: 0.9763 - val_loss: 2.0449 - val_accuracy: 0.9222
Epoch 7/10
438/438 [==============================] - 204s 466ms/step - loss: 0.1043 - accuracy: 0.9794 - val_loss: 2.303

In [ ]:
time.sleep(180)

In [38]:
n=16
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

16
Epoch 1/10
465/465 [==============================] - 221s 469ms/step - loss: 0.1419 - accuracy: 0.9695 - val_loss: 0.0708 - val_accuracy: 0.9819
Epoch 2/10
465/465 [==============================] - 217s 467ms/step - loss: 0.1100 - accuracy: 0.9742 - val_loss: 0.3770 - val_accuracy: 0.9576
Epoch 3/10
465/465 [==============================] - 214s 460ms/step - loss: 0.1035 - accuracy: 0.9759 - val_loss: 0.9946 - val_accuracy: 0.9617
Epoch 4/10
465/465 [==============================] - 195s 418ms/step - loss: 0.1101 - accuracy: 0.9747 - val_loss: 0.9960 - val_accuracy: 0.9589
Epoch 5/10
465/465 [==============================] - 194s 418ms/step - loss: 0.0922 - accuracy: 0.9787 - val_loss: 1.4989 - val_accuracy: 0.9597
Epoch 6/10
465/465 [==============================] - 195s 419ms/step - loss: 0.1065 - accuracy: 0.9763 - val_loss: 1.7676 - val_accuracy: 0.9597
Epoch 7/10
465/465 [==============================] - 194s 418ms/step - loss: 0.0930 - accuracy: 0.9806 - val_loss: 2.104

In [39]:
n=17
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1 

17
Epoch 1/10
493/493 [==============================] - 224s 450ms/step - loss: 0.1421 - accuracy: 0.9694 - val_loss: 0.1946 - val_accuracy: 0.9793
Epoch 2/10
493/493 [==============================] - 225s 457ms/step - loss: 0.0981 - accuracy: 0.9749 - val_loss: 0.3848 - val_accuracy: 0.9653
Epoch 3/10
493/493 [==============================] - 225s 456ms/step - loss: 0.1068 - accuracy: 0.9742 - val_loss: 0.8398 - val_accuracy: 0.9610
Epoch 4/10
493/493 [==============================] - 227s 461ms/step - loss: 0.1029 - accuracy: 0.9779 - val_loss: 0.9156 - val_accuracy: 0.9574
Epoch 5/10
493/493 [==============================] - 226s 458ms/step - loss: 0.1044 - accuracy: 0.9768 - val_loss: 1.2697 - val_accuracy: 0.9612
Epoch 6/10
493/493 [==============================] - 227s 460ms/step - loss: 0.1099 - accuracy: 0.9758 - val_loss: 2.0468 - val_accuracy: 0.9625
Epoch 7/10
493/493 [==============================] - 214s 434ms/step - loss: 0.1150 - accuracy: 0.9763 - val_loss: 1.837

In [40]:
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

18
Epoch 1/10
520/520 [==============================] - 219s 418ms/step - loss: 0.1521 - accuracy: 0.9645 - val_loss: 0.0818 - val_accuracy: 0.9860
Epoch 2/10
520/520 [==============================] - 216s 416ms/step - loss: 0.1184 - accuracy: 0.9705 - val_loss: 0.3537 - val_accuracy: 0.9727
Epoch 3/10
520/520 [==============================] - 217s 417ms/step - loss: 0.1150 - accuracy: 0.9737 - val_loss: 0.3276 - val_accuracy: 0.9701
Epoch 4/10
520/520 [==============================] - 231s 444ms/step - loss: 0.1139 - accuracy: 0.9734 - val_loss: 0.8668 - val_accuracy: 0.9719
Epoch 5/10
520/520 [==============================] - 237s 456ms/step - loss: 0.1079 - accuracy: 0.9741 - val_loss: 0.8631 - val_accuracy: 0.9707
Epoch 6/10
520/520 [==============================] - 236s 454ms/step - loss: 0.1251 - accuracy: 0.9723 - val_loss: 1.1506 - val_accuracy: 0.9712
Epoch 7/10
520/520 [==============================] - 239s 460ms/step - loss: 0.1150 - accuracy: 0.9747 - val_loss: 0.725

In [41]:
n=19
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

19
Epoch 1/10
547/547 [==============================] - 252s 457ms/step - loss: 0.1968 - accuracy: 0.9563 - val_loss: 0.0263 - val_accuracy: 0.9944
Epoch 2/10
547/547 [==============================] - 252s 460ms/step - loss: 0.1392 - accuracy: 0.9645 - val_loss: 0.0404 - val_accuracy: 0.9903
Epoch 3/10
547/547 [==============================] - 249s 455ms/step - loss: 0.1348 - accuracy: 0.9665 - val_loss: 0.0380 - val_accuracy: 0.9926
Epoch 4/10
547/547 [==============================] - 250s 456ms/step - loss: 0.1369 - accuracy: 0.9676 - val_loss: 0.0203 - val_accuracy: 0.9954
Epoch 5/10
547/547 [==============================] - 248s 454ms/step - loss: 0.1235 - accuracy: 0.9697 - val_loss: 0.0286 - val_accuracy: 0.9934
Epoch 6/10
547/547 [==============================] - 239s 437ms/step - loss: 0.1415 - accuracy: 0.9701 - val_loss: 0.0211 - val_accuracy: 0.9957
Epoch 7/10
547/547 [==============================] - 241s 440ms/step - loss: 0.1268 - accuracy: 0.9723 - val_loss: 0.032

In [42]:
n=20
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

20
Epoch 1/10
552/552 [==============================] - 253s 455ms/step - loss: 0.1931 - accuracy: 0.9541 - val_loss: 0.0285 - val_accuracy: 0.9941
Epoch 2/10
552/552 [==============================] - 253s 459ms/step - loss: 0.1453 - accuracy: 0.9638 - val_loss: 0.0490 - val_accuracy: 0.9908
Epoch 3/10
552/552 [==============================] - 252s 457ms/step - loss: 0.1384 - accuracy: 0.9664 - val_loss: 0.1135 - val_accuracy: 0.9765
Epoch 4/10
552/552 [==============================] - 251s 455ms/step - loss: 0.1320 - accuracy: 0.9673 - val_loss: 0.0379 - val_accuracy: 0.9944
Epoch 5/10
552/552 [==============================] - 251s 454ms/step - loss: 0.1363 - accuracy: 0.9695 - val_loss: 0.0321 - val_accuracy: 0.9944
Epoch 6/10
552/552 [==============================] - 252s 457ms/step - loss: 0.1246 - accuracy: 0.9707 - val_loss: 0.0522 - val_accuracy: 0.9944
Epoch 7/10
552/552 [==============================] - 250s 453ms/step - loss: 0.1353 - accuracy: 0.9717 - val_loss: 0.059

In [43]:
new_model_nc_dir  = "D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_nc_e1"

i=0

for model in models_nc:
    model.save(new_model_nc_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_nc_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_nc_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_nc_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_nc_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_nc_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_nc_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_nc_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model_c2_aug_nc_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C2/gtsrb_model

In [ ]:
loading = False

models_nc = []

if loading:
    for i in range(20):
        model_nc_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_nc_e1"+str(i)
        print(model_nc_dir)
        model =utils.My_model(dataset,True,model_nc_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_nc.append(model)
       

In [44]:
try:
    del nc_values
    del top_images_by_nc
    del top_labels_by_nc
    del image_sets_nc
    del label_sets_nc
    del models_nc
except:
    print("Error")

In [45]:
gc.collect()

632102

## Evaluating

In [46]:
model_original.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [47]:
# Metrics using adversarial test
evaluate_lsa_0 = []
evaluate_dsa_0 = []
evaluate_nc_0 = []
evaluate_random_0 = []

evaluate_lsa_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_dsa_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_nc_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_random_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))

for model in models_lsa:
    evaluate_lsa_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_dsa:
    evaluate_dsa_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_random:
    evaluate_random_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_nc:
    evaluate_nc_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))

123/123 [==============================] - 8s 64ms/step - loss: 1.7023 - accuracy: 0.7031 - precision: 0.8576 - recall: 0.7071


NameError: name 'models_lsa' is not defined

In [ ]:
evaluate_lsa_0[0]=[0,0,0,0]
evaluate_dsa_0[0]=[0,0,0,0]
evaluate_random_0[0]=[0,0,0,0]
evaluate_nc_0[0]=[0,0,0,0]

In [ ]:
print(len(evaluate_lsa_0))
print(len(evaluate_dsa_0))
print(len(evaluate_random_0))
print(len(evaluate_nc_0))

In [ ]:
# Metrics using original test
evaluate_lsa_1 = []
evaluate_dsa_1 = []
evaluate_nc_1 = []
evaluate_random_1 = []

evaluate_lsa_1.append(model_original.evaluate(x_test,y_test))
evaluate_dsa_1.append(model_original.evaluate(x_test,y_test))
evaluate_nc_1.append(model_original.evaluate(x_test,y_test))
evaluate_random_1.append(model_original.evaluate(x_test,y_test))
print("lsa---")
for model in models_lsa:
    evaluate_lsa_1.append(model.evaluate(x_test,y_test))

print("dsa---")
    
for model in models_dsa:
    evaluate_dsa_1.append(model.evaluate(x_test,y_test))
    

print("random---")

for model in models_random:
    evaluate_random_1.append(model.evaluate(x_test,y_test))
    
print("nc---")

for model in models_nc:
    evaluate_nc_1.append(model.evaluate(x_test,y_test))


In [ ]:
evaluate_lsa_1[0]=[0,0,0,0]
evaluate_dsa_1[0]=[0,0,0,0]
evaluate_random_1[0]=[0,0,0,0]
evaluate_nc_1[0]=[0,0,0,0]

In [ ]:

print(len(evaluate_lsa_1))
print(len(evaluate_dsa_1))
print(len(evaluate_random_1))
print(len(evaluate_nc_1))

In [ ]:
type(evaluate_dsa_0[0][0])

In [ ]:
import pandas as pd

df_evaluate_lsa_0 = pd.DataFrame(np.array(evaluate_lsa_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_lsa_1 = pd.DataFrame(np.array(evaluate_lsa_1),columns=["loss","accuracy","precision","recall"])
df_evaluate_dsa_0 = pd.DataFrame(np.array(evaluate_dsa_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_dsa_1 = pd.DataFrame(np.array(evaluate_dsa_1),columns=["loss","accuracy","precision","recall"])

df_evaluate_random_0 = pd.DataFrame(np.array(evaluate_random_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_random_1 = pd.DataFrame(np.array(evaluate_random_1),columns=["loss","accuracy","precision","recall"])
df_evaluate_nc_0 = pd.DataFrame(np.array(evaluate_nc_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_nc_1 = pd.DataFrame(np.array(evaluate_nc_1),columns=["loss","accuracy","precision","recall"])


In [ ]:
# Original test set + adversarial set

metric ="accuracy"
accuracy_lsa_3 = (np.array(df_evaluate_lsa_0[metric])+np.array(df_evaluate_lsa_1[metric]))/2
accuracy_dsa_3 = (np.array(df_evaluate_dsa_0[metric])+np.array(df_evaluate_dsa_1[metric]))/2
accuracy_nc_3 = (np.array(df_evaluate_nc_0[metric])+np.array(df_evaluate_nc_1[metric]))/2
accuracy_random_3 = (np.array(df_evaluate_random_0[metric])+np.array(df_evaluate_random_1[metric]))/2


## Charts

In [ ]:
n_inputs = [1800*i for i in range(20)]
n_inputs.append(len(x_train_and_adversary))

In [ ]:
print(n_inputs)

In [ ]:
cd "C:/Users/fjdur/Desktop/gtsrb_graphs/e2_graphs_dec/"

In [ ]:
linestyles = ['solid','dotted','dashed','dashdot']
colors =['k','k','k','k']

In [ ]:
#configuration 1
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

#metric = "accuracy" # accuracy loss
my_metrics =["accuracy","loss","precision","recall"]

for metric in my_metrics:
    plt.clf()
    plt.plot(n_inputs,df_evaluate_lsa_0[metric],colors[0],linestyle=linestyles[0])
    plt.plot(n_inputs,df_evaluate_dsa_0[metric],colors[1],linestyle=linestyles[1])
    plt.plot(n_inputs,df_evaluate_random_0[metric],colors[2],linestyle=linestyles[2])
    plt.plot(n_inputs,df_evaluate_nc_0[metric],colors[3],linestyle=linestyles[3])


    legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



    plt.legend(handles=legend_elements)#
    plt.title(metric + " with test set of adversarial examples FGSM")
    plt.xlim([0, 37000])
    #plt.ylim([0, 1])
    plt.xlabel('number of inputs')
    plt.ylabel(metric)
    plt.savefig("gtsrb_c1_"+metric + "_0.svg")
    plt.show()

In [ ]:
#configuration 1
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

my_metrics =["accuracy","loss","precision","recall"]

for metric in my_metrics:
    
#metric = "accuracy" # accuracy loss
    plt.clf()
    plt.plot(n_inputs,df_evaluate_lsa_1[metric],colors[0],linestyle=linestyles[0])
    plt.plot(n_inputs,df_evaluate_dsa_1[metric],colors[1],linestyle=linestyles[1])
    plt.plot(n_inputs,df_evaluate_random_1[metric],colors[2],linestyle=linestyles[2])
    plt.plot(n_inputs,df_evaluate_nc_1[metric],colors[3],linestyle=linestyles[3])

    legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



    plt.legend(handles=legend_elements)#
    plt.title(metric + " with original set")
    plt.xlim([0, 37000])
    #plt.ylim([0, 1])
    plt.xlabel('number of inputs')
    plt.ylabel(metric)
    plt.savefig("gtsrb_c1_"+metric + "_1.svg")
    plt.show()

In [ ]:
#adversarial jsma test set
#configuration 3
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

plt.plot(n_inputs,accuracy_lsa_3,colors[0],linestyle=linestyles[0])
plt.plot(n_inputs,accuracy_dsa_3,colors[1],linestyle=linestyles[1])
plt.plot(n_inputs,accuracy_random_3,colors[2],linestyle=linestyles[2])
plt.plot(n_inputs,accuracy_nc_3,colors[3],linestyle=linestyles[3])


legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



plt.legend(handles=legend_elements)#
plt.title("accuracy with both sets")

print(n_inputs[np.argmax(accuracy_lsa_3)],accuracy_lsa_3.max())
print(n_inputs[np.argmax(accuracy_dsa_3)],accuracy_dsa_3.max())
print(n_inputs[np.argmax(accuracy_random_3)],accuracy_random_3.max())
print(n_inputs[np.argmax(accuracy_nc_3)],accuracy_nc_3.max())

plt.plot(n_inputs[np.argmax(accuracy_lsa_3)],accuracy_lsa_3.max(),'-kD')
plt.plot(n_inputs[np.argmax(accuracy_dsa_3)],accuracy_dsa_3.max(),'-ko')
plt.plot(n_inputs[np.argmax(accuracy_random_3)],accuracy_random_3.max(),'-kv')
plt.plot(n_inputs[np.argmax(accuracy_nc_3)],accuracy_nc_3.max(),'-kp')

plt.xlabel('number of inputs')
plt.ylabel('accuracy')
plt.xlim([0, 37000])
#plt.ylim([0, 1])

plt.savefig("gtsrb_c1_"+"accuracy" + "_both.svg")

plt.show()

## Saving models

In [ ]:
new_model_lsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_lsa_e2"
i=0

for model in models_lsa:
    model.save(new_model_lsa_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_dsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_dsa_e2"

i=0
for model in models_dsa:
    model.save(new_model_dsa_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_random_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_random_e2"

i=0
for model in models_random:
    model.save(new_model_random_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_nc_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_nc_e2"

i=0
for model in models_nc:
    model.save(new_model_nc_dir+"_"+str(i))
    i+=1